# Import Libraries

In [1170]:
params = {}

In [1171]:
import time
import json
import sys

from data_task_helpers import something

something()

from api_data_task_executioner.data_task_tools import assert_dte_tools_available, get_resolved_parameters_for_connection, initialise_data_task, find_json_arg  # noqa: E402


something
['/Users/davidsongikandi/Documents/Gikandi/Work/repo/duft-workspace-django/duft-server', '/Library/Frameworks/Python.framework/Versions/3.11/lib/python311.zip', '/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11', '/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/lib-dynload', '', '/Users/davidsongikandi/Documents/Gikandi/Work/repo/duft-workspace-django/.venv/lib/python3.11/site-packages', '/Users/davidsongikandi/Documents/Gikandi/Work/repo/duft-workspace-django/duft', '/Users/davidsongikandi/Documents/Gikandi/Work/repo/duft-workspace-django/duft-server']


In [1172]:
   
environment = initialise_data_task("QE Data Task Running", params=params)
params["name"] = params.get("customname", params.get("name", "No parameters given!"))
params["sleep_time"] = params.get("sleep_time", 0.2)

if not params:
    environment.log_error("No parameters given!")
    

[2024-11-20T17:59:53.508813] QE Data Task Running - INFO: QE Data Task Running initialised running in active mode
QE Data Task Running >>>: QE Data Task Running initialised running in active mode


# Import dependancies

In [1173]:
import jaydebeapi
import pandas as pd
from pandas import DataFrame
from sqlalchemy import create_engine, text
from jaydebeapi import Connection
import time
from datetime import datetime

# Set Source and Destinaton Database Connections 

In [1174]:
source_environment = get_resolved_parameters_for_connection("EPMS_Source")
destination_environment = get_resolved_parameters_for_connection("EPMS_Destination")

environment.log_message(f'Finished setting source and destination connections')

[2024-11-20T17:59:53.524103] QE Data Task Running - INFO: Finished setting source and destination connections
QE Data Task Running >>>: Finished setting source and destination connections


# Set Filemaker Driver Files

In [1175]:
driver_jar_path = "fmjdbc.jar"
driver_class = "com.filemaker.jdbc.Driver"

# Process File Maker Data

In [1176]:
def open_file_maker_connection():
    try:
        server = source_environment.get("server")
        username = source_environment.get("username")
        password = source_environment.get("password")
        return jaydebeapi.connect(driver_class, server, [username, password], driver_jar_path)
    except Exception as ex:
        return None


def close_file_maker_connection(con: Connection):
    if con is not None:
        con.close()


def fetch_file_maker_data(query: str, con: Connection):
    cursor = con.cursor()
    cursor.execute(query)
    data_rows = cursor.fetchall()
    cursor.close()
    return data_rows

# Open connection to FM

In [1177]:
fm_connection = open_file_maker_connection()

# Create dim_patient Data Frame

In [1178]:
patient_confs = {
    "query": "SELECT  r.name, d.name, f.name, f.code, p.id, p.sex, p.dob, p.maritalStatus, p.patientStatus, p.clientCode, pharmacyNumber, pmtctNumber, nameFirst, nameLast, phoneCellNumber, phoneAltNumber, p.ageYear, resCurrentAddTown, resCurrentAddConstituency, resCurrentAddStreet, resPermanentAddTown, resPermanentAddConstituency, resPermanentAddStreet, idFacilityCurrent, tsName, tsCellPhoneNumber, tsSecondName, tsSecondCellPhoneNumber,  homeBasedCareOrg, homeBasedCareCode,  deathDate  FROM pat p LEFT join fac f on p.idFacilityCurrent = f.id  LEFT join region r on f.idregion=r.id LEFT join district d on f.idDistrict = d.id",
    "cols": ['region','district','current_facility', 'facility_code','client_id', 'sex', 'date_of_birth', 'marital_status','patient_status','client_code', 'pharmacy_code','pmtct_number', 'first_name','last_name','contact_number','alt_contact_number','age','current_town','current_constituency','current_street', 'permanent_town','permanent_constituency','permanent_street','id_facility_current', 'ts_name', 'ts_cell_phone_number', 'ts_second_name', 'ts_second_cell_phone_number', 'cbart_cargs_name', 'cbart_cargs_code', 'death_date']
}

patient_query = patient_confs.get("query")
patient_cols = patient_confs.get("cols")

patient_data = fetch_file_maker_data(patient_query, fm_connection)
dim_pat_df = pd.DataFrame(patient_data, columns=patient_cols)

dim_pat_df['date_of_birth'] = pd.to_datetime(dim_pat_df['date_of_birth'], errors='coerce')
dim_pat_df['first_name'] = 'FName'
dim_pat_df['last_name'] = 'LName'
dim_pat_df['contact_number'] = '123456'
dim_pat_df['alt_contact_number'] = '123456'
dim_pat_df['ts_name'] = 'TS_Name'
dim_pat_df['ts_second_name'] = 'TS_Second_Name'
dim_pat_df['ts_cell_phone_number'] = 'TS_Phone'
dim_pat_df['ts_second_cell_phone_number'] = 'TS_Second_Phone'

dim_pat_df['sex'] = dim_pat_df['sex'].fillna('Unknown')

dim_pat_df.head()

environment.log_message(f'Finished extracting client data')


[2024-11-20T17:59:57.682773] QE Data Task Running - INFO: Finished extracting client data
QE Data Task Running >>>: Finished extracting client data


# Get patient latest transfer status

In [1179]:
fact_tsfr_confs = {
    "query": "SELECT idpatient,  status, \"date\" FROM tsfr",
    "cols": ['client_id' ,'transfer_status', 'transfer_date']
}

fact_tsfr_query = fact_tsfr_confs.get("query")
fact_tsfr_cols = fact_tsfr_confs.get("cols")

fact_tsfr_data = fetch_file_maker_data(fact_tsfr_query, fm_connection)
fact_tsfr_df = pd.DataFrame(fact_tsfr_data, columns=fact_tsfr_cols)

fact_tsfr_df = fact_tsfr_df.dropna(subset=['transfer_date'])

fact_tsfr_df = fact_tsfr_df.loc[fact_tsfr_df.groupby('client_id')['transfer_date'].idxmin()].reset_index(drop=True)
fact_tsfr_df.head()

fact_tsfr_df.head(100)

environment.log_message(f'Finished extracting client transfer status data')

[2024-11-20T17:59:58.520222] QE Data Task Running - INFO: Finished extracting client transfer status data
QE Data Task Running >>>: Finished extracting client transfer status data


# Add transfer status to patient

In [1180]:
dim_pat_df = pd.merge(dim_pat_df, fact_tsfr_df, on='client_id', how='left')

dim_pat_df.head()

,region,district,current_facility,facility_code,client_id,sex,date_of_birth,marital_status,patient_status,client_code,...,id_facility_current,ts_name,ts_cell_phone_number,ts_second_name,ts_second_cell_phone_number,cbart_cargs_name,cbart_cargs_code,death_date,transfer_status,transfer_date
0,!Karas,!Nami≠Nûs,Luderitz District Hospital,10221,F2E91406-846B-7D40-9544-C522FE691720,Female,1971-04-04,Single,Active ART,None,...,EF916F03-F628-4E36-9659-889E4443BCB9,TS_Name,TS_Phone,TS_Second_Name,TS_Second_Phone,None,None,None,In,0001-01-01
1,None,None,Unknown,99999,EEB0C17D-193C-9A46-B0F3-447E59D74CB2,Male,1960-06-06,Married,Transferred Out,None,...,931237D1-CBAB-49DF-8844-67EB50DA522D,TS_Name,TS_Phone,TS_Second_Name,TS_Second_Phone,None,None,None,Out,2013-02-05
2,None,None,Unknown,99999,00BFC838-2A1C-AA41-9389-E480215EBE3D,Female,1969-04-14,Single,Transferred Out,None,...,931237D1-CBAB-49DF-8844-67EB50DA522D,TS_Name,TS_Phone,TS_Second_Name,TS_Second_Phone,None,None,None,Out,2008-09-01
3,None,None,Unknown,99999,6C708B5E-FF4F-064E-9597-230BFF374297,Male,1970-01-01,Married,Transferred Out,None,...,931237D1-CBAB-49DF-8844-67EB50DA522D,TS_Name,TS_Phone,TS_Second_Name,TS_Second_Phone,None,None,None,Out,2008-07-07
4,Ohangwena,Engela,Onamukulo Clinic,10389,792BA050-2DAF-DA48-B918-786C5B751C76,Female,1970-09-22,Single,Transferred Out,None,...,BF8397FF-AC48-4007-95D8-72CEAF62701B,TS_Name,TS_Phone,TS_Second_Name,TS_Second_Phone,None,,None,In,0001-01-01


# Get first Weight and WHO Stage

In [1181]:
fact_meas_confs = {
    "query": "SELECT idpatient, weight, whoStage,\"Date\" FROM Meas m",
    "cols": ['client_id', 'weight','who_stage', 'creation_date']
}

fact_meas_query = fact_meas_confs.get("query")
fact_meas_cols = fact_meas_confs.get("cols")

fact_meas_data = fetch_file_maker_data(fact_meas_query, fm_connection)
fact_meas_df = pd.DataFrame(fact_meas_data, columns=fact_meas_cols)

fact_meas_df = fact_meas_df.dropna(subset=['creation_date'])

fact_meas_df = fact_meas_df.loc[fact_meas_df.groupby('client_id')['creation_date'].idxmin()].reset_index(drop=True)
fact_meas_df.head()

,client_id,weight,who_stage,creation_date
0,0029CA07-AB4E-7D4D-99E2-6DBE978B6FF0,66.8,1,2020-11-26
1,005BC7AF-7312-4249-82CA-6DF3ED4D70B1,112.0,1,2021-01-07
2,007A9BF5-2F37-8A4F-8EF6-84400DEFAABF,76.1,1,2020-03-18
3,00A6338D-BB1C-7F48-9D36-0213212F78C0,84.0,1,2021-03-31
4,00ADFB9C-5F27-1344-BEB7-94B33DF7C03C,82.0,1,2021-04-28


# Create fact_hiv_diagnosis Data Frame

In [1182]:
fact_hiv_diagnosis_confs = {
    "query": "SELECT idpatient, hivconfirmationdate, hivconfirmatoryresultsdate, hivconfirmatoryresultstype, idfacilitycreate, fullDisclosureDAte FROM Cd",
    "cols": ['client_id', 'hiv_confirmation_date','hiv_confirmatory_result_date', 'hiv_confirmatory_result_type', 'hiv_diagnosis_facility_id','full_disclosure_date']
}

fact_hiv_diagnosis_query = fact_hiv_diagnosis_confs.get("query")
fact_hiv_diagnosis_cols = fact_hiv_diagnosis_confs.get("cols")

fact_hiv_diagnosis_data = fetch_file_maker_data(fact_hiv_diagnosis_query, fm_connection)
fact_hiv_diagnosis_df = pd.DataFrame(fact_hiv_diagnosis_data, columns=fact_hiv_diagnosis_cols)

fact_hiv_diagnosis_df['hiv_confirmation_date'] = pd.to_datetime(fact_hiv_diagnosis_df['hiv_confirmation_date'], errors='coerce')
fact_hiv_diagnosis_df['hiv_confirmatory_result_date'] = pd.to_datetime(fact_hiv_diagnosis_df['hiv_confirmatory_result_date'], errors='coerce')

fact_hiv_diagnosis_df = fact_hiv_diagnosis_df.dropna(subset=['hiv_confirmation_date'])

fact_hiv_diagnosis_df = fact_hiv_diagnosis_df.loc[fact_hiv_diagnosis_df.groupby('client_id')['hiv_confirmation_date'].idxmin()].reset_index(drop=True)

fact_hiv_diagnosis_df.head()

environment.log_message(f'Finished extracting HIV diagnosis data')

[2024-11-20T18:00:14.447586] QE Data Task Running - INFO: Finished extracting HIV diagnosis data
QE Data Task Running >>>: Finished extracting HIV diagnosis data


# Add Initial weight and WHO Stage to fact_hiv_diagnosis

In [1183]:
fact_hiv_diagnosis_df = pd.merge(fact_hiv_diagnosis_df, fact_meas_df[['client_id','weight', 'who_stage']], on='client_id', how='left')

fact_hiv_diagnosis_df.head()

,client_id,hiv_confirmation_date,hiv_confirmatory_result_date,hiv_confirmatory_result_type,hiv_diagnosis_facility_id,full_disclosure_date,weight,who_stage
0,0029CA07-AB4E-7D4D-99E2-6DBE978B6FF0,2012-01-27,NaT,None,EF916F03-F628-4E36-9659-889E4443BCB9,None,66.8,1
1,002BF341-6105-604C-B9EF-77340C27C371,2009-12-21,NaT,None,EF916F03-F628-4E36-9659-889E4443BCB9,None,NaN,NaN
2,00303D7F-B74F-1C41-9A62-E82C59B8B5E8,2005-12-15,NaT,None,EF916F03-F628-4E36-9659-889E4443BCB9,None,NaN,NaN
3,00411EBF-278C-3247-A7A1-FED6115B2F8D,2010-12-31,NaT,None,3510BB1C-0F10-4CA9-9960-718C5C7D5A2E,None,NaN,NaN
4,005BC7AF-7312-4249-82CA-6DF3ED4D70B1,2009-11-12,NaT,None,EF916F03-F628-4E36-9659-889E4443BCB9,None,112.0,1


# Add dim_patient, fact_hiv_diagnosis to fact_sentinel_event dataframe

In [1184]:
fact_sentinel_event_df = pd.merge(dim_pat_df, fact_hiv_diagnosis_df, on='client_id', how='left')

fact_sentinel_event_df.head()

,region,district,current_facility,facility_code,client_id,sex,date_of_birth,marital_status,patient_status,client_code,...,death_date,transfer_status,transfer_date,hiv_confirmation_date,hiv_confirmatory_result_date,hiv_confirmatory_result_type,hiv_diagnosis_facility_id,full_disclosure_date,weight,who_stage
0,!Karas,!Nami≠Nûs,Luderitz District Hospital,10221,F2E91406-846B-7D40-9544-C522FE691720,Female,1971-04-04,Single,Active ART,None,...,None,In,0001-01-01,2003-10-01,NaT,None,EF916F03-F628-4E36-9659-889E4443BCB9,None,98.3,1
1,None,None,Unknown,99999,EEB0C17D-193C-9A46-B0F3-447E59D74CB2,Male,1960-06-06,Married,Transferred Out,None,...,None,Out,2013-02-05,2001-01-01,NaT,None,EF916F03-F628-4E36-9659-889E4443BCB9,None,NaN,NaN
2,None,None,Unknown,99999,00BFC838-2A1C-AA41-9389-E480215EBE3D,Female,1969-04-14,Single,Transferred Out,None,...,None,Out,2008-09-01,1996-01-01,NaT,None,EF916F03-F628-4E36-9659-889E4443BCB9,None,NaN,NaN
3,None,None,Unknown,99999,6C708B5E-FF4F-064E-9597-230BFF374297,Male,1970-01-01,Married,Transferred Out,None,...,None,Out,2008-07-07,2002-09-11,NaT,None,EF916F03-F628-4E36-9659-889E4443BCB9,None,NaN,NaN
4,Ohangwena,Engela,Onamukulo Clinic,10389,792BA050-2DAF-DA48-B918-786C5B751C76,Female,1970-09-22,Single,Transferred Out,None,...,None,In,0001-01-01,2003-11-21,NaT,None,EF916F03-F628-4E36-9659-889E4443BCB9,None,74.5,1


# Create hiv_enrolment dataframe 

In [1185]:
fact_hiv_enrolment_confs = {
    "query": "SELECT c.idpatient, c.idfacilitycreate, c.hivenrolleddate, c.disclosureenrollmentdate, c.arteligiblereason, c.artstartdate, p.artnumber, p.artnumber, p.artnumberlegacy, c.idfacilityARTStart FROM pat p LEFT JOIN Cd c ON p.id=c.idpatient",
    "cols": ['client_id', 'hiv_enrollment_facility_id', 'hiv_enrollment_date', 'hiv_disclosure_enrollment_date','art_eligible_reason','art_start_date', 'quantum_number' , 'art_number', 'art_number_legacy', 'arv_initiating_facility']
}


fact_hiv_enrolment_query = fact_hiv_enrolment_confs.get("query")
fact_hiv_enrolment_cols = fact_hiv_enrolment_confs.get("cols")

fact_hiv_enrolment_data = fetch_file_maker_data(fact_hiv_enrolment_query, fm_connection)
fact_hiv_enrolment_df = pd.DataFrame(fact_hiv_enrolment_data, columns=fact_hiv_enrolment_cols)

fact_hiv_enrolment_df['hiv_enrollment_date'] = pd.to_datetime(fact_hiv_enrolment_df['hiv_enrollment_date'], errors='coerce')
fact_hiv_enrolment_df['hiv_disclosure_enrollment_date'] = pd.to_datetime(fact_hiv_enrolment_df['hiv_disclosure_enrollment_date'], errors='coerce')
fact_hiv_enrolment_df['art_start_date'] = pd.to_datetime(fact_hiv_enrolment_df['art_start_date'], errors='coerce')

fact_hiv_enrolment_df = fact_hiv_enrolment_df.dropna(subset=['hiv_enrollment_date'])

fact_hiv_enrolment_df = fact_hiv_enrolment_df.loc[fact_hiv_enrolment_df.groupby('client_id')['hiv_enrollment_date'].idxmin()].reset_index(drop=True)

fact_hiv_enrolment_df.head()

environment.log_message(f'Finished extracting HIV enrolment data')

[2024-11-20T18:00:15.274095] QE Data Task Running - INFO: Finished extracting HIV enrolment data
QE Data Task Running >>>: Finished extracting HIV enrolment data


# Add fact_hiv_enrolment to fact_sentinel_event dataframe

In [1186]:
fact_sentinel_event_df = pd.merge(fact_sentinel_event_df, fact_hiv_enrolment_df, on='client_id', how='left')
fact_sentinel_event_df.head()

,region,district,current_facility,facility_code,client_id,sex,date_of_birth,marital_status,patient_status,client_code,...,who_stage,hiv_enrollment_facility_id,hiv_enrollment_date,hiv_disclosure_enrollment_date,art_eligible_reason,art_start_date,quantum_number,art_number,art_number_legacy,arv_initiating_facility
0,!Karas,!Nami≠Nûs,Luderitz District Hospital,10221,F2E91406-846B-7D40-9544-C522FE691720,Female,1971-04-04,Single,Active ART,None,...,1,EF916F03-F628-4E36-9659-889E4443BCB9,2003-10-01,NaT,Clinical & CD4,2003-10-17,10221-1003-1,10221-1003-1,109100300001,EF916F03-F628-4E36-9659-889E4443BCB9
1,None,None,Unknown,99999,EEB0C17D-193C-9A46-B0F3-447E59D74CB2,Male,1960-06-06,Married,Transferred Out,None,...,NaN,EF916F03-F628-4E36-9659-889E4443BCB9,2003-10-30,NaT,CD4 # or %,2003-11-10,10221-1003-2,10221-1003-2,109100300002,EF916F03-F628-4E36-9659-889E4443BCB9
2,None,None,Unknown,99999,00BFC838-2A1C-AA41-9389-E480215EBE3D,Female,1969-04-14,Single,Transferred Out,None,...,NaN,EF916F03-F628-4E36-9659-889E4443BCB9,2003-11-01,NaT,CD4 # or %,2003-12-17,10221-1103-3,10221-1103-3,109110300003,EF916F03-F628-4E36-9659-889E4443BCB9
3,None,None,Unknown,99999,6C708B5E-FF4F-064E-9597-230BFF374297,Male,1970-01-01,Married,Transferred Out,None,...,NaN,EF916F03-F628-4E36-9659-889E4443BCB9,2003-11-12,NaT,CD4 # or %,2003-12-17,10221-1103-4,10221-1103-4,109110300004,EF916F03-F628-4E36-9659-889E4443BCB9
4,Ohangwena,Engela,Onamukulo Clinic,10389,792BA050-2DAF-DA48-B918-786C5B751C76,Female,1970-09-22,Single,Transferred Out,None,...,1,EF916F03-F628-4E36-9659-889E4443BCB9,2003-11-21,NaT,CD4 # or %,2003-12-30,10221-1103-5,10221-1103-5,109110300005,EF916F03-F628-4E36-9659-889E4443BCB9


# Get first and last visit ids per patient

In [1187]:
fact_visits_confs = {
    "query": "SELECT id, idpatient, visitDate FROM Fup",
    "cols": ['id', 'client_id', 'visit_date' ]
}

fact_visits_query = fact_visits_confs.get("query")
fact_visits_cols = fact_visits_confs.get("cols")

fact_visits_data = fetch_file_maker_data(fact_visits_query, fm_connection)
fact_visits_df = pd.DataFrame(fact_visits_data, columns=fact_visits_cols)

latest_visits_df = fact_visits_df.loc[fact_visits_df.groupby('client_id')['visit_date'].idxmax()]

first_visits_df = fact_visits_df.loc[fact_visits_df.groupby('client_id')['visit_date'].idxmin()]

# Create first_fact_visits dataframe 

In [1188]:
fact_visits_confs = {
    "query": "SELECT DISTINCT f.idpatient, f.idfacilitycreate, f.visitDate FROM Fup f",
    "cols": ['client_id', 'visit_facility_id', 'visit_date']
}

fact_visits_query = fact_visits_confs.get("query")
fact_visits_cols = fact_visits_confs.get("cols")

ids = first_visits_df['id'].tolist()

ids_str = ','.join(f"'{id_}'" for id_ in ids)

fact_visits_query_updated = f"{fact_visits_query} WHERE f.id IN ({ids_str})"

fact_visits_data = fetch_file_maker_data(fact_visits_query_updated, fm_connection)
fact_first_visit_df = pd.DataFrame(fact_visits_data, columns=fact_visits_cols)

fact_first_visit_df['visit_date'] = pd.to_datetime(fact_first_visit_df['visit_date'], errors='coerce')

fact_first_visit_df.head() 

,client_id,visit_facility_id,visit_date
0,0029CA07-AB4E-7D4D-99E2-6DBE978B6FF0,EF916F03-F628-4E36-9659-889E4443BCB9,2012-02-23
1,002BF341-6105-604C-B9EF-77340C27C371,EF916F03-F628-4E36-9659-889E4443BCB9,2010-01-13
2,00303D7F-B74F-1C41-9A62-E82C59B8B5E8,EF916F03-F628-4E36-9659-889E4443BCB9,2007-09-26
3,00411EBF-278C-3247-A7A1-FED6115B2F8D,EF916F03-F628-4E36-9659-889E4443BCB9,2012-02-22
4,005BC7AF-7312-4249-82CA-6DF3ED4D70B1,EF916F03-F628-4E36-9659-889E4443BCB9,2009-11-20


# Add first visit info to fact_sentinel_event

In [1189]:
columns_to_add = {
    'visit_date': 'first_visit_date', 
    'visit_facility_id': 'first_visit_facility_id'
}

fact_first_visit_df = fact_first_visit_df[list(columns_to_add.keys()) + ['client_id']].rename(columns=columns_to_add)

fact_sentinel_event_df = pd.merge(fact_sentinel_event_df, fact_first_visit_df, on='client_id', how='left')
fact_sentinel_event_df.head()

,region,district,current_facility,facility_code,client_id,sex,date_of_birth,marital_status,patient_status,client_code,...,hiv_enrollment_date,hiv_disclosure_enrollment_date,art_eligible_reason,art_start_date,quantum_number,art_number,art_number_legacy,arv_initiating_facility,first_visit_date,first_visit_facility_id
0,!Karas,!Nami≠Nûs,Luderitz District Hospital,10221,F2E91406-846B-7D40-9544-C522FE691720,Female,1971-04-04,Single,Active ART,None,...,2003-10-01,NaT,Clinical & CD4,2003-10-17,10221-1003-1,10221-1003-1,109100300001,EF916F03-F628-4E36-9659-889E4443BCB9,2006-03-29,EF916F03-F628-4E36-9659-889E4443BCB9
1,None,None,Unknown,99999,EEB0C17D-193C-9A46-B0F3-447E59D74CB2,Male,1960-06-06,Married,Transferred Out,None,...,2003-10-30,NaT,CD4 # or %,2003-11-10,10221-1003-2,10221-1003-2,109100300002,EF916F03-F628-4E36-9659-889E4443BCB9,2006-08-24,EF916F03-F628-4E36-9659-889E4443BCB9
2,None,None,Unknown,99999,00BFC838-2A1C-AA41-9389-E480215EBE3D,Female,1969-04-14,Single,Transferred Out,None,...,2003-11-01,NaT,CD4 # or %,2003-12-17,10221-1103-3,10221-1103-3,109110300003,EF916F03-F628-4E36-9659-889E4443BCB9,2004-04-26,EF916F03-F628-4E36-9659-889E4443BCB9
3,None,None,Unknown,99999,6C708B5E-FF4F-064E-9597-230BFF374297,Male,1970-01-01,Married,Transferred Out,None,...,2003-11-12,NaT,CD4 # or %,2003-12-17,10221-1103-4,10221-1103-4,109110300004,EF916F03-F628-4E36-9659-889E4443BCB9,2006-07-12,EF916F03-F628-4E36-9659-889E4443BCB9
4,Ohangwena,Engela,Onamukulo Clinic,10389,792BA050-2DAF-DA48-B918-786C5B751C76,Female,1970-09-22,Single,Transferred Out,None,...,2003-11-21,NaT,CD4 # or %,2003-12-30,10221-1103-5,10221-1103-5,109110300005,EF916F03-F628-4E36-9659-889E4443BCB9,2006-10-16,EF916F03-F628-4E36-9659-889E4443BCB9


# Create fact_last_visit dataframe 

In [1190]:
fact_visits_confs = {
    "query": "SELECT DISTINCT f.idpatient, f.idfacilitycreate, f.visitDate, f.followupDate, f.scheduledDate, f.careModel, f.pregnantstatus, f.Breastfeeding, f.pregnantlmp, f.pregnantedd, f.ctxadherence, f.arvadherence, cc.treatment, cc.dateTreatment, labResult, pf.stiGenitalUlcers, pf.stiVaginalUrethralDischarge, pf.stiScreenResult, f.oiDetail, f.oiOther, f.tbScreenResult FROM Fup f LEFT JOIN patFup pf on f.id=pf.id LEFT JOIN cc ON f.id=cc.idFollowUp",
    "cols": ['client_id', 'visit_facility_id', 'visit_date', 'next_visit_date', 'scheduled_visit_date', 'care_model', 'pregnancy_status', 'breast_feeding', 'lmp', 'edd', 'ctx_adherence', 'arv_adherence', 'cc_treatment_type','cc_treatment_date','cc_results', 'genital_ulcers','vaginal_urethral_discharge','sti_screening_result', 'oi', 'oi_other','tb_screen_result' ]
}

fact_visits_query = fact_visits_confs.get("query")
fact_visits_cols = fact_visits_confs.get("cols")

ids = latest_visits_df['id'].tolist()

ids_str = ','.join(f"'{id_}'" for id_ in ids)

fact_visits_query_updated = f"{fact_visits_query} WHERE f.id IN ({ids_str})"

fact_visits_data = fetch_file_maker_data(fact_visits_query_updated, fm_connection)
fact_last_visit_df = pd.DataFrame(fact_visits_data, columns=fact_visits_cols)

fact_last_visit_df['visit_date'] = pd.to_datetime(fact_last_visit_df['visit_date'], errors='coerce')
fact_last_visit_df['next_visit_date'] = pd.to_datetime(fact_last_visit_df['next_visit_date'], errors='coerce')

fact_last_visit_df['care_model'] = fact_last_visit_df['care_model'].fillna('Unknown')

fact_last_visit_df.head() 

,client_id,visit_facility_id,visit_date,next_visit_date,scheduled_visit_date,care_model,pregnancy_status,breast_feeding,lmp,edd,...,arv_adherence,cc_treatment_type,cc_treatment_date,cc_results,genital_ulcers,vaginal_urethral_discharge,sti_screening_result,oi,oi_other,tb_screen_result
0,0029CA07-AB4E-7D4D-99E2-6DBE978B6FF0,EF916F03-F628-4E36-9659-889E4443BCB9,2024-08-22,2024-12-30,2024-08-22,ART Clinic,None,None,None,None,...,Good,None,None,None,No,No,Completed,None,None,Negative
1,002BF341-6105-604C-B9EF-77340C27C371,EF916F03-F628-4E36-9659-889E4443BCB9,2014-03-17,2014-04-17,2014-03-17,Unknown,None,None,None,None,...,G,None,None,None,None,None,None,None,None,Incomplete
2,00303D7F-B74F-1C41-9A62-E82C59B8B5E8,EF916F03-F628-4E36-9659-889E4443BCB9,2010-04-28,2010-06-03,2010-04-28,Unknown,None,None,None,None,...,G,None,None,None,None,None,None,None,None,Incomplete
3,00411EBF-278C-3247-A7A1-FED6115B2F8D,EF916F03-F628-4E36-9659-889E4443BCB9,2012-02-22,2012-03-26,None,Unknown,None,None,None,None,...,G,None,None,None,None,None,None,None,RTI,Incomplete
4,005BC7AF-7312-4249-82CA-6DF3ED4D70B1,EF916F03-F628-4E36-9659-889E4443BCB9,2024-05-30,2024-11-28,2024-05-30,ART Clinic,No,No,None,None,...,Good,None,None,None,No,No,Completed,None,None,Negative


# Add last visit info to fact_sentinel_event

In [1191]:
columns_to_add = {
    'visit_date': 'last_visit_date',
    'next_visit_date': 'last_next_visit_date', 
    'scheduled_visit_date': 'last_scheduled_visit_date', 
    'pregnancy_status': 'last_pregnancy_status', 
    'breast_feeding': 'last_breast_feeding', 
    'lmp': 'last_lmp', 
    'edd': 'last_edd',
    'care_model': 'last_care_model',
    'visit_facility_id': 'last_visit_facility_id',
    'cc_treatment_type':'last_cc_treatment_type',
    'cc_treatment_date': 'last_cc_treatment_date',
    'cc_results': 'last_cc_results',
    'oi':'last_oi', 
    'oi_other': 'last_oi_other',
    'tb_screen_result': 'last_tb_screen_result'
}

fact_last_visit_df = fact_last_visit_df[list(columns_to_add.keys()) + ['client_id']].rename(columns=columns_to_add)

last_refresh_date = datetime.strptime(datetime.now().strftime('%Y-%m-%d'), '%Y-%m-%d')

print(f'last_refresh_date: {last_refresh_date}')

fact_sentinel_event_df = pd.merge(fact_sentinel_event_df, fact_last_visit_df, on='client_id', how='left')

fact_sentinel_event_df['last_visit_duration'] = (fact_sentinel_event_df['last_next_visit_date'].dt.year - fact_sentinel_event_df['last_visit_date'].dt.year) * 12 + (fact_sentinel_event_df['last_next_visit_date'].dt.month - fact_sentinel_event_df['last_visit_date'].dt.month)

fact_sentinel_event_df['iit_date'] = fact_sentinel_event_df['last_next_visit_date'] + pd.Timedelta(days=29)

def diff_in_months(date1, date2):
    if pd.isnull(date1) or pd.isnull(date2):
        return None
    return (date1.year - date2.year) * 12 + date1.month - date2.month

fact_sentinel_event_df['months_since_hiv_confirmed'] = fact_sentinel_event_df.apply(lambda row: diff_in_months(last_refresh_date, row['hiv_confirmation_date']), axis=1)

fact_sentinel_event_df['iit_duration'] = (pd.Timestamp('today').normalize() - fact_sentinel_event_df['iit_date']).dt.days

fact_sentinel_event_df['last_PBFW'] = fact_sentinel_event_df.apply(
    lambda row: 'Pregnant' if row['last_pregnancy_status'] == 'Yes' and row['sex'] == 'Female' else 'Breast feeding' if row['last_breast_feeding'] == 'Yes' and row['sex'] == 'Female' else None, 
    axis=1
)


fact_sentinel_event_df['last_PBFW_status'] = fact_sentinel_event_df.apply(
    lambda row: 'Yes' if pd.notna(row['last_PBFW']) and row['sex'] == 'Female' else 'No' if row['sex'] == 'Female' else None,
    axis=1
)

fact_sentinel_event_df.head()

environment.log_message(f'Finished extracting client visits data')

last_refresh_date: 2024-11-20 00:00:00
[2024-11-20T18:00:37.323490] QE Data Task Running - INFO: Finished extracting client visits data
QE Data Task Running >>>: Finished extracting client visits data


# Create patient regimen dataframe

In [1192]:
fact_reg_confs = {
    "query": "SELECT idpatient, idfacilitycreate, \"date\", type,line, code, dosage,duration, reason FROM rgm",
    "cols": ['client_id', 'ti_facility_id', 'regimen_date', 'regimen_type', 'regimen_line' ,'regimen', 'regimen_dosage','regimen_duration', 'regimen_reason']
}

fact_reg_query = fact_reg_confs.get("query")
fact_reg_cols = fact_reg_confs.get("cols")

fact_reg_data = fetch_file_maker_data(fact_reg_query, fm_connection)
fact_reg_df = pd.DataFrame(fact_reg_data, columns=fact_reg_cols)

fact_reg_df['regimen_date'] = pd.to_datetime(fact_reg_df['regimen_date'], errors='coerce')

fact_reg_df = fact_reg_df.dropna(subset=['client_id', 'regimen_date'])

fact_reg_df = fact_reg_df.loc[fact_reg_df.groupby('client_id')['regimen_date'].idxmax()]

fact_reg_df.head()

environment.log_message(f'Finished extracting client regimen data')

[2024-11-20T18:00:39.067451] QE Data Task Running - INFO: Finished extracting client regimen data
QE Data Task Running >>>: Finished extracting client regimen data


# Add regimen info to fact_sentinel_event

In [1193]:
columns_to_add = ['client_id','regimen_line' ,'regimen','regimen_date']

fact_reg_df = fact_reg_df[columns_to_add]

fact_sentinel_event_df = pd.merge(fact_sentinel_event_df, fact_reg_df, on='client_id', how='left')

fact_sentinel_event_df = fact_sentinel_event_df.rename(columns={
    'regimen_line': 'last_regimen_line',
    'regimen': 'last_regimen',
    'regimen_date': 'last_regimen_date'
})

fact_sentinel_event_df.head()

,region,district,current_facility,facility_code,client_id,sex,date_of_birth,marital_status,patient_status,client_code,...,last_tb_screen_result,last_visit_duration,iit_date,months_since_hiv_confirmed,iit_duration,last_PBFW,last_PBFW_status,last_regimen_line,last_regimen,last_regimen_date
0,!Karas,!Nami≠Nûs,Luderitz District Hospital,10221,F2E91406-846B-7D40-9544-C522FE691720,Female,1971-04-04,Single,Active ART,None,...,Negative,6.0,2025-04-01,253.0,-132.0,None,No,1,TDF/3TC/DTG(99/00),2020-09-07
1,None,None,Unknown,99999,EEB0C17D-193C-9A46-B0F3-447E59D74CB2,Male,1960-06-06,Married,Transferred Out,None,...,Incomplete,3.0,2012-09-14,286.0,4450.0,None,None,1,TDF/3TC/EFV (1f),2009-02-23
2,None,None,Unknown,99999,00BFC838-2A1C-AA41-9389-E480215EBE3D,Female,1969-04-14,Single,Transferred Out,None,...,Incomplete,NaN,NaT,346.0,NaN,None,No,1,AZT/3TC/EFV (1b / 4d),2008-08-12
3,None,None,Unknown,99999,6C708B5E-FF4F-064E-9597-230BFF374297,Male,1970-01-01,Married,Transferred Out,None,...,Incomplete,NaN,NaT,266.0,NaN,None,None,1,D4T/3TC/EFV (1d / 4c),2003-12-17
4,Ohangwena,Engela,Onamukulo Clinic,10389,792BA050-2DAF-DA48-B918-786C5B751C76,Female,1970-09-22,Single,Transferred Out,None,...,Incomplete,12.0,2024-01-20,252.0,305.0,None,No,1,TDF/3TC/DTG(99/00),2020-06-10


# Create fact_ti dataframe 

In [1194]:
fact_ti_confs = {
    "query": "SELECT idpatient, idfacilitycreate, interruptionDate,interruptionReason, interruptionReasonOther, restartDate,duration FROM ti",
    "cols": ['client_id', 'ti_facility_id', 'art_interruption_date', 'art_interruption_reason', 'art_interruption_reason_other' ,'art_restart_date', 'interruption_duration']
}

fact_ti_query = fact_ti_confs.get("query")
fact_ti_cols = fact_ti_confs.get("cols")

fact_ti_data = fetch_file_maker_data(fact_ti_query, fm_connection)
fact_ti_df = pd.DataFrame(fact_ti_data, columns=fact_ti_cols)

fact_ti_df['art_restart_date'] = pd.to_datetime(fact_ti_df['art_restart_date'], errors='coerce')

fact_ti_df = fact_ti_df.dropna(subset=['client_id', 'art_interruption_date'])

fact_ti_df = fact_ti_df.loc[fact_ti_df.groupby('client_id')['art_interruption_date'].idxmax()]

fact_ti_df.head()

environment.log_message(f'Finished extracting client treatment interruption data')

[2024-11-20T18:00:39.219495] QE Data Task Running - INFO: Finished extracting client treatment interruption data
QE Data Task Running >>>: Finished extracting client treatment interruption data


# Add treatment interription info to fact_sentinel_event

In [1195]:
columns_to_add = ['client_id', 'art_interruption_date', 'art_interruption_reason', 'art_interruption_reason_other' ,'art_restart_date']

fact_ti_df = fact_ti_df[columns_to_add]

fact_sentinel_event_df = pd.merge(fact_sentinel_event_df, fact_ti_df, on='client_id', how='left')
fact_sentinel_event_df.head()

,region,district,current_facility,facility_code,client_id,sex,date_of_birth,marital_status,patient_status,client_code,...,iit_duration,last_PBFW,last_PBFW_status,last_regimen_line,last_regimen,last_regimen_date,art_interruption_date,art_interruption_reason,art_interruption_reason_other,art_restart_date
0,!Karas,!Nami≠Nûs,Luderitz District Hospital,10221,F2E91406-846B-7D40-9544-C522FE691720,Female,1971-04-04,Single,Active ART,None,...,-132.0,None,No,1,TDF/3TC/DTG(99/00),2020-09-07,NaN,NaN,NaN,NaT
1,None,None,Unknown,99999,EEB0C17D-193C-9A46-B0F3-447E59D74CB2,Male,1960-06-06,Married,Transferred Out,None,...,4450.0,None,None,1,TDF/3TC/EFV (1f),2009-02-23,NaN,NaN,NaN,NaT
2,None,None,Unknown,99999,00BFC838-2A1C-AA41-9389-E480215EBE3D,Female,1969-04-14,Single,Transferred Out,None,...,NaN,None,No,1,AZT/3TC/EFV (1b / 4d),2008-08-12,NaN,NaN,NaN,NaT
3,None,None,Unknown,99999,6C708B5E-FF4F-064E-9597-230BFF374297,Male,1970-01-01,Married,Transferred Out,None,...,NaN,None,None,1,D4T/3TC/EFV (1d / 4c),2003-12-17,NaN,NaN,NaN,NaT
4,Ohangwena,Engela,Onamukulo Clinic,10389,792BA050-2DAF-DA48-B918-786C5B751C76,Female,1970-09-22,Single,Transferred Out,None,...,305.0,None,No,1,TDF/3TC/DTG(99/00),2020-06-10,NaN,NaN,NaN,NaT


# Create fact_tbt dataframe 

In [1196]:
fact_tbt_confs = {
    "query": "SELECT idpatient, idfacilitycreate, category, startDate, regimen, duration, stopDateExpected, stopDateActual , registrationNumber , site, siteDetail , CASE  WHEN outcome = '1' THEN 'Cured' WHEN outcome = '2' THEN 'Treatment complete' WHEN outcome = '3' THEN 'Died' WHEN outcome = '4' THEN 'Failure' WHEN outcome = '5' THEN 'Lost to follow-up' WHEN outcome = '6' THEN 'Not evaluated' END AS outcome, devCreationTimestamp  FROM tbt",
    "cols": ['client_id', 'tbt_facility_id', 'tbt_category', 'tbt_start_date', 'tbt_regimen', 'tbt_duration' ,'tbt_expected_stop_date', 'tbt_actual_stop_date', 'tbt_registration_number', 'tbt_site', 'tbt_site_detail', 'tbt_outcome', 'tbt_outcome_date']
}

fact_tbt_query = fact_tbt_confs.get("query")
fact_tbt_cols = fact_tbt_confs.get("cols")

fact_tbt_data = fetch_file_maker_data(fact_tbt_query, fm_connection)
fact_tbt_df = pd.DataFrame(fact_tbt_data, columns=fact_tbt_cols)

fact_tbt_df['tbt_outcome_date'] = pd.to_datetime(fact_tbt_df['tbt_outcome_date'])
fact_tbt_df['tbt_start_date'] = pd.to_datetime(fact_tbt_df['tbt_start_date'])
fact_tbt_df['tbt_outcome_date'] = fact_tbt_df['tbt_outcome_date'].dt.date

fact_tbt_df.loc[fact_tbt_df['tbt_outcome'].isnull(), 'tbt_outcome_date'] = None

fact_tbt_df = fact_tbt_df.dropna(subset=['tbt_start_date'])

fact_tbt_df = fact_tbt_df.loc[fact_tbt_df.groupby('client_id')['tbt_start_date'].idxmax()]

fact_tbt_df.head()

environment.log_message(f'Finished extracting client tbt data')

[2024-11-20T18:00:39.402785] QE Data Task Running - INFO: Finished extracting client tbt data
QE Data Task Running >>>: Finished extracting client tbt data


# Add tbt info to fact_sentinel_event

In [1197]:
columns_to_add = ['client_id', 'tbt_facility_id', 'tbt_category', 'tbt_start_date', 'tbt_regimen', 'tbt_duration' ,'tbt_expected_stop_date', 'tbt_actual_stop_date', 'tbt_registration_number', 'tbt_site', 'tbt_site_detail', 'tbt_outcome', 'tbt_outcome_date']

fact_tbt_df = fact_tbt_df[columns_to_add]

fact_sentinel_event_df = pd.merge(fact_sentinel_event_df, fact_tbt_df, on='client_id', how='left')
fact_sentinel_event_df.head()


,region,district,current_facility,facility_code,client_id,sex,date_of_birth,marital_status,patient_status,client_code,...,tbt_start_date,tbt_regimen,tbt_duration,tbt_expected_stop_date,tbt_actual_stop_date,tbt_registration_number,tbt_site,tbt_site_detail,tbt_outcome,tbt_outcome_date
0,!Karas,!Nami≠Nûs,Luderitz District Hospital,10221,F2E91406-846B-7D40-9544-C522FE691720,Female,1971-04-04,Single,Active ART,None,...,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,None,None,Unknown,99999,EEB0C17D-193C-9A46-B0F3-447E59D74CB2,Male,1960-06-06,Married,Transferred Out,None,...,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,None,None,Unknown,99999,00BFC838-2A1C-AA41-9389-E480215EBE3D,Female,1969-04-14,Single,Transferred Out,None,...,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,None,None,Unknown,99999,6C708B5E-FF4F-064E-9597-230BFF374297,Male,1970-01-01,Married,Transferred Out,None,...,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Ohangwena,Engela,Onamukulo Clinic,10389,792BA050-2DAF-DA48-B918-786C5B751C76,Female,1970-09-22,Single,Transferred Out,None,...,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Create fact_tpt dataframe 

In [1198]:
fact_tpt_confs = {
    "query": "SELECT idpatient, idfacilitycreate, iptRegimen, startDate, duration, stopDateExpected, stopDateActual , CASE  WHEN stopReason = '1' THEN 'Completed TPT' WHEN stopReason = '2' THEN 'Cough' WHEN stopReason = '3' THEN 'Confirmed TB' WHEN stopReason = '4' THEN 'Hepatitis' WHEN stopReason = '5' THEN 'Neuropathy' WHEN stopReason = '6' THEN 'Poor Adherence' WHEN stopReason = '7' THEN 'Medicine Out Of Stock' WHEN stopReason = '8' THEN stopReasonOther ELSE '' END AS stopReason, status, adherenceDetail FROM ipt",
    "cols": ['client_id', 'tpt_facility_id', 'tpt_regimen', 'tpt_start_date', 'tpt_duration' ,'tpt_expected_stop_date', 'tpt_actual_stop_date', 'tpt_stop_reason', 'tpt_status', 'tpt_adherance']
}

fact_tpt_query = fact_tpt_confs.get("query")
fact_tpt_cols = fact_tpt_confs.get("cols")

fact_tpt_data = fetch_file_maker_data(fact_tpt_query, fm_connection)
fact_tpt_df = pd.DataFrame(fact_tpt_data, columns=fact_tpt_cols)

fact_tpt_df = fact_tpt_df.dropna(subset=['client_id', 'tpt_start_date'])

fact_tpt_df = fact_tpt_df.loc[fact_tpt_df.groupby('client_id')['tpt_start_date'].idxmax()]

fact_tpt_df['tpt_start_date'] = pd.to_datetime(fact_tpt_df['tpt_start_date'], errors='coerce')
fact_tpt_df['tpt_actual_stop_date'] = pd.to_datetime(fact_tpt_df['tpt_actual_stop_date'], errors='coerce')
fact_tpt_df['tpt_expected_stop_date'] = pd.to_datetime(fact_tpt_df['tpt_expected_stop_date'], errors='coerce')

fact_tpt_df['tpt_duration_two'] = fact_tpt_df.apply(
    lambda row: (row['tpt_expected_stop_date'] - row['tpt_start_date']).days 
                if pd.notna(row['tpt_expected_stop_date']) and pd.notna(row['tpt_start_date']) 
                else None,
    axis=1
)

fact_tpt_df.loc[
    (fact_tpt_df['tpt_regimen'] == '3H') & (fact_tpt_df['tpt_start_date'] > '2020-08-01'),
    'tpt_regimen'
] = '3HP'

fact_tpt_df.head()

environment.log_message(f'Finished extracting client tpt data')

[2024-11-20T18:00:40.040359] QE Data Task Running - INFO: Finished extracting client tpt data
QE Data Task Running >>>: Finished extracting client tpt data


In [1199]:
# Check for invalid dates in the 'tpt_start_date' column
invalid_dates = fact_tpt_df[pd.to_datetime(fact_tpt_df['tpt_expected_stop_date'], errors='coerce').isna()]

# Display rows with invalid dates
invalid_dates.head()

,client_id,tpt_facility_id,tpt_regimen,tpt_start_date,tpt_duration,tpt_expected_stop_date,tpt_actual_stop_date,tpt_stop_reason,tpt_status,tpt_adherance,tpt_duration_two


# Add tpt info to fact_sentinel_event

In [1200]:
columns_to_add = ['client_id', 'tpt_facility_id', 'tpt_regimen', 'tpt_start_date', 'tpt_duration', 'tpt_duration_two' ,'tpt_expected_stop_date', 'tpt_actual_stop_date', 'tpt_stop_reason', 'tpt_status', 'tpt_adherance']

fact_tpt_df = fact_tpt_df[columns_to_add]

fact_sentinel_event_df = pd.merge(fact_sentinel_event_df, fact_tpt_df, on='client_id', how='left')

fact_sentinel_event_df.head()

,region,district,current_facility,facility_code,client_id,sex,date_of_birth,marital_status,patient_status,client_code,...,tpt_facility_id,tpt_regimen,tpt_start_date,tpt_duration,tpt_duration_two,tpt_expected_stop_date,tpt_actual_stop_date,tpt_stop_reason,tpt_status,tpt_adherance
0,!Karas,!Nami≠Nûs,Luderitz District Hospital,10221,F2E91406-846B-7D40-9544-C522FE691720,Female,1971-04-04,Single,Active ART,None,...,EF916F03-F628-4E36-9659-889E4443BCB9,9H,2010-09-28,9.0,273.0,2011-06-28,2011-06-30,Completed TPT,Stopped,Good
1,None,None,Unknown,99999,EEB0C17D-193C-9A46-B0F3-447E59D74CB2,Male,1960-06-06,Married,Transferred Out,None,...,EF916F03-F628-4E36-9659-889E4443BCB9,None,2011-03-02,0.0,0.0,2011-03-02,2011-08-29,Completed TPT,Stopped,None
2,None,None,Unknown,99999,00BFC838-2A1C-AA41-9389-E480215EBE3D,Female,1969-04-14,Single,Transferred Out,None,...,NaN,NaN,NaT,NaN,NaN,NaT,NaT,NaN,NaN,NaN
3,None,None,Unknown,99999,6C708B5E-FF4F-064E-9597-230BFF374297,Male,1970-01-01,Married,Transferred Out,None,...,NaN,NaN,NaT,NaN,NaN,NaT,NaT,NaN,NaN,NaN
4,Ohangwena,Engela,Onamukulo Clinic,10389,792BA050-2DAF-DA48-B918-786C5B751C76,Female,1970-09-22,Single,Transferred Out,None,...,EF916F03-F628-4E36-9659-889E4443BCB9,6H,2011-05-19,6.0,184.0,2011-11-19,2011-11-23,Completed TPT,Stopped,Good


# Compute TPT Type

In [1201]:
fact_sentinel_event_df['tpt_type'] = fact_sentinel_event_df.apply(
    lambda row: "9H" if pd.notna(row['tpt_start_date']) and row['tpt_duration_two'] >= 250 else
                ("6H" if pd.notna(row['tpt_start_date']) and row['tpt_duration_two'] >= 168 else
                 ("3HP" if pd.notna(row['tpt_start_date']) and row['tpt_start_date'] >= pd.to_datetime("2020-08-01") and 
                          pd.notna(row['tpt_expected_stop_date']) and row['tpt_duration_two'] >= 77 else
                  ("3HP" if row['tpt_regimen'] in ["3H", "3HP"] and row['tpt_start_date'] >= pd.to_datetime("2020-08-01") else None))),
    axis=1
)

fact_sentinel_event_df['tpt_type'] = fact_sentinel_event_df.apply(
    lambda row: "3HP" if pd.isna(row['tpt_type']) and 
                          row['tpt_regimen'] in ["3H", "3HP"] and 
                          row['tpt_start_date'] >= pd.to_datetime("2020-08-01") else row['tpt_type'],
    axis=1
)

fact_sentinel_event_df.head()

,region,district,current_facility,facility_code,client_id,sex,date_of_birth,marital_status,patient_status,client_code,...,tpt_regimen,tpt_start_date,tpt_duration,tpt_duration_two,tpt_expected_stop_date,tpt_actual_stop_date,tpt_stop_reason,tpt_status,tpt_adherance,tpt_type
0,!Karas,!Nami≠Nûs,Luderitz District Hospital,10221,F2E91406-846B-7D40-9544-C522FE691720,Female,1971-04-04,Single,Active ART,None,...,9H,2010-09-28,9.0,273.0,2011-06-28,2011-06-30,Completed TPT,Stopped,Good,9H
1,None,None,Unknown,99999,EEB0C17D-193C-9A46-B0F3-447E59D74CB2,Male,1960-06-06,Married,Transferred Out,None,...,None,2011-03-02,0.0,0.0,2011-03-02,2011-08-29,Completed TPT,Stopped,None,None
2,None,None,Unknown,99999,00BFC838-2A1C-AA41-9389-E480215EBE3D,Female,1969-04-14,Single,Transferred Out,None,...,NaN,NaT,NaN,NaN,NaT,NaT,NaN,NaN,NaN,None
3,None,None,Unknown,99999,6C708B5E-FF4F-064E-9597-230BFF374297,Male,1970-01-01,Married,Transferred Out,None,...,NaN,NaT,NaN,NaN,NaT,NaT,NaN,NaN,NaN,None
4,Ohangwena,Engela,Onamukulo Clinic,10389,792BA050-2DAF-DA48-B918-786C5B751C76,Female,1970-09-22,Single,Transferred Out,None,...,6H,2011-05-19,6.0,184.0,2011-11-19,2011-11-23,Completed TPT,Stopped,Good,6H


# Compute tpt_status_two_outcome

In [1202]:
def compute_tpt_status_two_outcome(row):
    # Define constants
    tpt_complete_other = 146
    tpt_complete_3HP = 70
    tpt_on_tx_3H = 113
    tpt_on_tx_other = 240

    # Calculate the time differences for various conditions
    start_to_actual_stop = (row['tpt_actual_stop_date'] - row['tpt_start_date']).days if pd.notna(row['tpt_actual_stop_date']) and pd.notna(row['tpt_start_date']) else None
    start_to_last_refresh = (last_refresh_date - row['tpt_start_date']).days if pd.notna(row['tpt_start_date']) else None

    if pd.isna(row['tpt_start_date']):
        return "No TPT Documentation"
    elif pd.notna(row['tpt_start_date']) and pd.notna(row['tpt_actual_stop_date']) and row['tpt_start_date'] >= row['tpt_actual_stop_date']:
        return "Wrong TPT Dates"
    elif pd.isna(row['tpt_start_date']) and pd.notna(row['tpt_actual_stop_date']):
        return "Wrong TPT Dates"
    elif pd.notna(row['tpt_start_date']) and pd.isna(row['tpt_regimen']) and pd.isna(row['tpt_expected_stop_date']) and pd.isna(row['tpt_actual_stop_date']):
        return "TPT regimen is blank"
    elif pd.notna(row['tpt_start_date']) and row['tpt_expected_stop_date'] == row['tpt_start_date'] and pd.isna(row['tpt_regimen']) and pd.isna(row['tpt_actual_stop_date']):
        return "TPT regimen is blank"
    elif pd.notna(row['tpt_start_date']) and row['tpt_expected_stop_date'] == row['tpt_start_date'] and pd.notna(row['tpt_regimen']) and row['tpt_regimen'] == "Other" and pd.isna(row['tpt_actual_stop_date']):
        return "TPT regimen is other"
    elif pd.isna(row['tpt_type']) and pd.notna(row['tpt_actual_stop_date']) and start_to_actual_stop >= tpt_complete_other:
        return "6-month TPT Completed but TPT Type is unknown"
    elif pd.isna(row['tpt_type']) and pd.isna(row['tpt_actual_stop_date']) and start_to_last_refresh >= tpt_complete_other:
        return "6-month TPT Completed but TPT Type is unknown"
    elif row['tpt_status'] == "Active" and row['tpt_type'] == "3HP" and pd.isna(row['tpt_actual_stop_date']) and start_to_last_refresh < tpt_on_tx_3H:
     return "Still on 3HP"
    elif row['tpt_status'] == "Active" and row['tpt_type'] == "6H" and pd.isna(row['tpt_actual_stop_date']) and start_to_last_refresh < tpt_on_tx_other:
        return "Still on 6H"
    elif row['tpt_status'] == "Active" and row['tpt_type'] == "9H" and pd.isna(row['tpt_actual_stop_date']) and start_to_last_refresh < tpt_on_tx_other:
        return "Still on 9H"
    elif row['tpt_type'] == "3HP" and pd.isna(row['tpt_actual_stop_date']) and start_to_last_refresh >= tpt_on_tx_3H:
     return "3HP Completed but TPT Stop Date is blank"
    elif row['tpt_type'] == "3HP" and start_to_actual_stop >= tpt_complete_3HP:
     return "3HP Completed"
    elif row['tpt_type'] == "3HP" and pd.notna(row['tpt_actual_stop_date']) and start_to_actual_stop < tpt_complete_3HP:
     return "3HP Stopped Before Completion"
    elif row['tpt_type'] == "6H" and pd.notna(row['tpt_actual_stop_date']) and start_to_actual_stop >= tpt_complete_other:
        return "6H Completed"
    elif row['tpt_type'] == "6H" and pd.isna(row['tpt_actual_stop_date']) and start_to_last_refresh >= tpt_on_tx_other:
        return "6H Completed but TPT Stop Date is blank"
    elif row['tpt_type'] == "6H" and pd.notna(row['tpt_actual_stop_date']) and start_to_actual_stop < tpt_complete_other:
        return "6H stopped Before 6-month Completion"
    elif row['tpt_type'] == "9H" and pd.notna(row['tpt_actual_stop_date']) and start_to_actual_stop >= tpt_complete_other:
        return "6-month of 9H Completed"
    elif row['tpt_type'] == "9H" and pd.isna(row['tpt_actual_stop_date']) and start_to_last_refresh >= tpt_on_tx_other:
        return "6-months of 9H Completed but TPT Stop Date is blank"
    elif row['tpt_type'] == "9H" and pd.notna(row['tpt_actual_stop_date']) and start_to_actual_stop < tpt_complete_other:
        return "9H stopped Before 6-month Completion"
    elif pd.isna(row['tpt_type']) and pd.notna(row['tpt_actual_stop_date']) and start_to_actual_stop < tpt_complete_other:
        return "TPT stopped before 6-month completion & TPT Type is unknown"
    elif pd.notna(row['tpt_start_date']) and row['tpt_start_date'] == row['tpt_expected_stop_date'] and pd.isna(row['tpt_actual_stop_date']):
        return "TPT regimen is blank"
    else:
        return None


fact_sentinel_event_df['tpt_status_two_outcome'] = fact_sentinel_event_df.apply(compute_tpt_status_two_outcome, axis=1)

fact_sentinel_event_df.head()

,region,district,current_facility,facility_code,client_id,sex,date_of_birth,marital_status,patient_status,client_code,...,tpt_start_date,tpt_duration,tpt_duration_two,tpt_expected_stop_date,tpt_actual_stop_date,tpt_stop_reason,tpt_status,tpt_adherance,tpt_type,tpt_status_two_outcome
0,!Karas,!Nami≠Nûs,Luderitz District Hospital,10221,F2E91406-846B-7D40-9544-C522FE691720,Female,1971-04-04,Single,Active ART,None,...,2010-09-28,9.0,273.0,2011-06-28,2011-06-30,Completed TPT,Stopped,Good,9H,6-month of 9H Completed
1,None,None,Unknown,99999,EEB0C17D-193C-9A46-B0F3-447E59D74CB2,Male,1960-06-06,Married,Transferred Out,None,...,2011-03-02,0.0,0.0,2011-03-02,2011-08-29,Completed TPT,Stopped,None,None,6-month TPT Completed but TPT Type is unknown
2,None,None,Unknown,99999,00BFC838-2A1C-AA41-9389-E480215EBE3D,Female,1969-04-14,Single,Transferred Out,None,...,NaT,NaN,NaN,NaT,NaT,NaN,NaN,NaN,None,No TPT Documentation
3,None,None,Unknown,99999,6C708B5E-FF4F-064E-9597-230BFF374297,Male,1970-01-01,Married,Transferred Out,None,...,NaT,NaN,NaN,NaT,NaT,NaN,NaN,NaN,None,No TPT Documentation
4,Ohangwena,Engela,Onamukulo Clinic,10389,792BA050-2DAF-DA48-B918-786C5B751C76,Female,1970-09-22,Single,Transferred Out,None,...,2011-05-19,6.0,184.0,2011-11-19,2011-11-23,Completed TPT,Stopped,Good,6H,6H Completed


# Create fact_lab dataframe 

In [1203]:
fact_lab_confs = {
    "query": "SELECT l.idpatient, l.idfacilitycreate, l.orderDate, l.result, l.resultDate, lm.name, l.status FROM lab l INNER JOIN LabMaster lm ON lm.id=l.idtest",
    "cols": ['client_id', 'lab_facility_id', 'order_date', 'lab_result', 'result_date', 'test_name','status' ]
}

fact_lab_query = fact_lab_confs.get("query")
fact_lab_cols = fact_lab_confs.get("cols")

fact_lab_data = fetch_file_maker_data(fact_lab_query, fm_connection)
fact_lab_df = pd.DataFrame(fact_lab_data, columns=fact_lab_cols)

fact_lab_df['order_date'] = pd.to_datetime(fact_lab_df['order_date'], errors='coerce')
fact_lab_df['result_date'] = pd.to_datetime(fact_lab_df['result_date'], errors='coerce')


fact_lab_df.head()

environment.log_message(f'Finished extracting client lab data')

[2024-11-20T18:01:07.843390] QE Data Task Running - INFO: Finished extracting client lab data
QE Data Task Running >>>: Finished extracting client lab data


# Create fact_first_viral_load dataframe

In [1204]:
fact_first_viral_load_tests_df = fact_lab_df[fact_lab_df['test_name'] == 'Viral Load']
fact_first_viral_load_tests_df = fact_first_viral_load_tests_df.sort_values('order_date').groupby('client_id').first().reset_index()

def categorize_result(result):
    if result < 40:
        return 'Undetectable'
    elif 40 <= result < 1000:
        return 'LLV'
    else:
        return 'Unsuppressed'

fact_first_viral_load_tests_df['category'] = fact_first_viral_load_tests_df['lab_result'].apply(categorize_result)

fact_first_viral_load_tests_df.head()

,client_id,lab_facility_id,order_date,lab_result,result_date,test_name,status,category
0,0029CA07-AB4E-7D4D-99E2-6DBE978B6FF0,EF916F03-F628-4E36-9659-889E4443BCB9,2012-10-01,28.08,2012-10-01,Viral Load,Complete,Undetectable
1,002BF341-6105-604C-B9EF-77340C27C371,EF916F03-F628-4E36-9659-889E4443BCB9,2010-08-25,3098.22,2010-08-25,Viral Load,Complete,Unsuppressed
2,00303D7F-B74F-1C41-9A62-E82C59B8B5E8,EF916F03-F628-4E36-9659-889E4443BCB9,2008-10-02,40.00,2008-10-02,Viral Load,Complete,LLV
3,005BC7AF-7312-4249-82CA-6DF3ED4D70B1,EF916F03-F628-4E36-9659-889E4443BCB9,2011-08-22,437.70,2011-08-22,Viral Load,Complete,LLV
4,007A9BF5-2F37-8A4F-8EF6-84400DEFAABF,EF916F03-F628-4E36-9659-889E4443BCB9,2008-08-04,267.69,2008-08-04,Viral Load,Complete,LLV


# Create fact_last_viral_load dataframe

In [1205]:
fact_last_viral_load_tests_df = fact_lab_df[fact_lab_df['test_name'] == 'Viral Load']
fact_last_viral_load_tests_df = fact_last_viral_load_tests_df.sort_values('order_date').groupby('client_id').last().reset_index()

def categorize_result(result):
    if result < 40:
        return 'Undetectable'
    elif 40 <= result < 1000:
        return 'LLV'
    else:
        return 'Unsuppressed'

fact_last_viral_load_tests_df['category'] = fact_last_viral_load_tests_df['lab_result'].apply(categorize_result)

fact_last_viral_load_tests_df.head()

,client_id,lab_facility_id,order_date,lab_result,result_date,test_name,status,category
0,0029CA07-AB4E-7D4D-99E2-6DBE978B6FF0,EF916F03-F628-4E36-9659-889E4443BCB9,2023-12-20,19.00,2023-12-20,Viral Load,Complete,Undetectable
1,002BF341-6105-604C-B9EF-77340C27C371,EF916F03-F628-4E36-9659-889E4443BCB9,2014-01-06,162177.73,2014-01-06,Viral Load,Complete,Unsuppressed
2,00303D7F-B74F-1C41-9A62-E82C59B8B5E8,EF916F03-F628-4E36-9659-889E4443BCB9,2009-04-23,56.00,2009-04-23,Viral Load,Complete,LLV
3,005BC7AF-7312-4249-82CA-6DF3ED4D70B1,EF916F03-F628-4E36-9659-889E4443BCB9,2024-04-25,0.00,2024-04-25,Viral Load,Complete,Undetectable
4,007A9BF5-2F37-8A4F-8EF6-84400DEFAABF,EF916F03-F628-4E36-9659-889E4443BCB9,2024-03-14,0.00,2024-03-14,Viral Load,Complete,Undetectable


# Add first viral load info to fact_sentinel_event

In [1206]:
columns_to_add = {
    'order_date': 'first_viral_load_order_date',
    'result_date': 'first_viral_load_result_date', 
    'lab_result': 'first_viral_load_result',
    'lab_facility_id': 'first_viral_load_facility_id',
    'category': 'first_viral_load_result_category',
    'status': 'first_viral_load_result_status',
}

fact_first_viral_load_tests_df = fact_first_viral_load_tests_df[list(columns_to_add.keys()) + ['client_id']].rename(columns=columns_to_add)

fact_sentinel_event_df = pd.merge(fact_sentinel_event_df, fact_first_viral_load_tests_df, on='client_id', how='left')

fact_sentinel_event_df.head()

,region,district,current_facility,facility_code,client_id,sex,date_of_birth,marital_status,patient_status,client_code,...,tpt_status,tpt_adherance,tpt_type,tpt_status_two_outcome,first_viral_load_order_date,first_viral_load_result_date,first_viral_load_result,first_viral_load_facility_id,first_viral_load_result_category,first_viral_load_result_status
0,!Karas,!Nami≠Nûs,Luderitz District Hospital,10221,F2E91406-846B-7D40-9544-C522FE691720,Female,1971-04-04,Single,Active ART,None,...,Stopped,Good,9H,6-month of 9H Completed,2008-09-15,2008-09-15,4736.41,EF916F03-F628-4E36-9659-889E4443BCB9,Unsuppressed,Complete
1,None,None,Unknown,99999,EEB0C17D-193C-9A46-B0F3-447E59D74CB2,Male,1960-06-06,Married,Transferred Out,None,...,Stopped,None,None,6-month TPT Completed but TPT Type is unknown,2008-08-20,2008-08-20,544.08,EF916F03-F628-4E36-9659-889E4443BCB9,LLV,Complete
2,None,None,Unknown,99999,00BFC838-2A1C-AA41-9389-E480215EBE3D,Female,1969-04-14,Single,Transferred Out,None,...,NaN,NaN,None,No TPT Documentation,NaT,NaT,NaN,NaN,NaN,NaN
3,None,None,Unknown,99999,6C708B5E-FF4F-064E-9597-230BFF374297,Male,1970-01-01,Married,Transferred Out,None,...,NaN,NaN,None,No TPT Documentation,NaT,NaT,NaN,NaN,NaN,NaN
4,Ohangwena,Engela,Onamukulo Clinic,10389,792BA050-2DAF-DA48-B918-786C5B751C76,Female,1970-09-22,Single,Transferred Out,None,...,Stopped,Good,6H,6H Completed,2009-04-23,2009-04-23,13464.50,EF916F03-F628-4E36-9659-889E4443BCB9,Unsuppressed,Complete


# Add last viral load info to fact_sentinel_event

In [1207]:
columns_to_add = {
    'order_date': 'last_viral_load_order_date',
    'result_date': 'last_viral_load_result_date', 
    'lab_result': 'last_viral_load_result',
    'lab_facility_id': 'last_viral_load_facility_id',
    'category': 'last_viral_load_result_category',
    'status': 'last_viral_load_result_status',
}

fact_last_viral_load_tests_df = fact_last_viral_load_tests_df[list(columns_to_add.keys()) + ['client_id']].rename(columns=columns_to_add)

fact_sentinel_event_df = pd.merge(fact_sentinel_event_df, fact_last_viral_load_tests_df, on='client_id', how='left')

# Add duration of ART in months

In [1208]:
def diff_in_months(date1, date2):
    if pd.isnull(date1) or pd.isnull(date2):
        return None
    return (date1.year - date2.year) * 12 + date1.month - date2.month

fact_sentinel_event_df['months_since_art_start'] = fact_sentinel_event_df.apply(lambda row: diff_in_months(last_refresh_date, row['art_start_date']), axis=1)

fact_sentinel_event_df['months_since_art_restart'] = fact_sentinel_event_df.apply(lambda row: diff_in_months(last_refresh_date, row['art_restart_date']), axis=1)

fact_sentinel_event_df.head()

,region,district,current_facility,facility_code,client_id,sex,date_of_birth,marital_status,patient_status,client_code,...,first_viral_load_result_category,first_viral_load_result_status,last_viral_load_order_date,last_viral_load_result_date,last_viral_load_result,last_viral_load_facility_id,last_viral_load_result_category,last_viral_load_result_status,months_since_art_start,months_since_art_restart
0,!Karas,!Nami≠Nûs,Luderitz District Hospital,10221,F2E91406-846B-7D40-9544-C522FE691720,Female,1971-04-04,Single,Active ART,None,...,Unsuppressed,Complete,2024-05-30,2024-05-30,29.00,EF916F03-F628-4E36-9659-889E4443BCB9,Undetectable,Complete,253.0,NaN
1,None,None,Unknown,99999,EEB0C17D-193C-9A46-B0F3-447E59D74CB2,Male,1960-06-06,Married,Transferred Out,None,...,LLV,Complete,2011-03-02,2011-03-02,946.87,EF916F03-F628-4E36-9659-889E4443BCB9,LLV,Complete,252.0,NaN
2,None,None,Unknown,99999,00BFC838-2A1C-AA41-9389-E480215EBE3D,Female,1969-04-14,Single,Transferred Out,None,...,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,251.0,NaN
3,None,None,Unknown,99999,6C708B5E-FF4F-064E-9597-230BFF374297,Male,1970-01-01,Married,Transferred Out,None,...,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,251.0,NaN
4,Ohangwena,Engela,Onamukulo Clinic,10389,792BA050-2DAF-DA48-B918-786C5B751C76,Female,1970-09-22,Single,Transferred Out,None,...,Unsuppressed,Complete,2022-01-27,2022-01-27,19.00,EF916F03-F628-4E36-9659-889E4443BCB9,Undetectable,Complete,251.0,NaN


# Add program status

In [1209]:
def classify_status(row):
    if (pd.notna(row['patient_status']) and
        pd.notna(row['art_start_date']) and
        row['patient_status'] == 'Active ART' and
        (last_refresh_date - pd.to_datetime(row['art_start_date'])).days <= 31 and
        (pd.isna(row['last_next_visit_date']) or pd.isna(row['last_visit_date']))):
        return row['patient_status']
    
    elif (pd.notna(row['patient_status']) and
          pd.isna(row['last_next_visit_date']) and
          (last_refresh_date - pd.to_datetime(row['last_visit_date'])).days <= 365 and
          row['patient_status'] == 'Active ART'):
        return row['patient_status']
    
    elif (pd.notna(row['patient_status']) and row['patient_status'] == 'Deceased'):
        return row['patient_status']
    
    elif (pd.notna(row['patient_status']) and row['patient_status'] == 'ART Stopped'):
        return row['patient_status']
    
    elif (pd.notna(row['patient_status']) and row['patient_status'] == 'Transferred Out'):
        return row['patient_status']
    
    elif (pd.notna(row['iit_date']) and
          pd.to_datetime(row['iit_date']) > last_refresh_date and
          (pd.notna(row['art_start_date']) or pd.notna(row['last_regimen']))):
        return 'Active ART'
    
    elif (pd.notna(row['iit_date']) and
          pd.to_datetime(row['iit_date']) <= last_refresh_date and
          (pd.notna(row['art_start_date']) or pd.notna(row['last_regimen']))):
        return 'IIT'
    
    else:
        return None
    
fact_sentinel_event_df['client_status'] = fact_sentinel_event_df.apply(classify_status, axis=1)

fact_sentinel_event_df['tx_curr'] = fact_sentinel_event_df['client_status'].apply(
    lambda status: 'Yes' if status == 'Active ART' else 'No'
)

fact_sentinel_event_df.head()

,region,district,current_facility,facility_code,client_id,sex,date_of_birth,marital_status,patient_status,client_code,...,last_viral_load_order_date,last_viral_load_result_date,last_viral_load_result,last_viral_load_facility_id,last_viral_load_result_category,last_viral_load_result_status,months_since_art_start,months_since_art_restart,client_status,tx_curr
0,!Karas,!Nami≠Nûs,Luderitz District Hospital,10221,F2E91406-846B-7D40-9544-C522FE691720,Female,1971-04-04,Single,Active ART,None,...,2024-05-30,2024-05-30,29.00,EF916F03-F628-4E36-9659-889E4443BCB9,Undetectable,Complete,253.0,NaN,Active ART,Yes
1,None,None,Unknown,99999,EEB0C17D-193C-9A46-B0F3-447E59D74CB2,Male,1960-06-06,Married,Transferred Out,None,...,2011-03-02,2011-03-02,946.87,EF916F03-F628-4E36-9659-889E4443BCB9,LLV,Complete,252.0,NaN,Transferred Out,No
2,None,None,Unknown,99999,00BFC838-2A1C-AA41-9389-E480215EBE3D,Female,1969-04-14,Single,Transferred Out,None,...,NaT,NaT,NaN,NaN,NaN,NaN,251.0,NaN,Transferred Out,No
3,None,None,Unknown,99999,6C708B5E-FF4F-064E-9597-230BFF374297,Male,1970-01-01,Married,Transferred Out,None,...,NaT,NaT,NaN,NaN,NaN,NaN,251.0,NaN,Transferred Out,No
4,Ohangwena,Engela,Onamukulo Clinic,10389,792BA050-2DAF-DA48-B918-786C5B751C76,Female,1970-09-22,Single,Transferred Out,None,...,2022-01-27,2022-01-27,19.00,EF916F03-F628-4E36-9659-889E4443BCB9,Undetectable,Complete,251.0,NaN,Transferred Out,No


# Add Upto date on TPT

In [1210]:
# Define constants
tpt_complete_other = 146
tpt_complete_3HP = 70

# Ensure the calculation happens for each row using apply
fact_sentinel_event_df['up_to_date_tpt'] = fact_sentinel_event_df.apply(
    lambda row: 1 if (
        # Check if tx_curr is 'Yes'
        row['tx_curr'] == 'Yes' and (
            # Condition 1: TPT completed for 3HP
            (pd.notna(row['tpt_actual_stop_date']) and pd.notna(row['tpt_start_date']) and
             (row['tpt_actual_stop_date'] - row['tpt_start_date']).days >= tpt_complete_3HP and
             row['tpt_type'] in ["3HP", "3H"]) or
            # Condition 2: TPT completed for other regimens
            (pd.notna(row['tpt_actual_stop_date']) and pd.notna(row['tpt_start_date']) and
             (row['tpt_actual_stop_date'] - row['tpt_start_date']).days >= tpt_complete_other and
             row['tpt_type'] in ["6HP", "6H", "9H", "9HP", None])  # None for not listed regimens
        )
    ) else 0,
    axis=1
)
fact_sentinel_event_df.head()

,region,district,current_facility,facility_code,client_id,sex,date_of_birth,marital_status,patient_status,client_code,...,last_viral_load_result_date,last_viral_load_result,last_viral_load_facility_id,last_viral_load_result_category,last_viral_load_result_status,months_since_art_start,months_since_art_restart,client_status,tx_curr,up_to_date_tpt
0,!Karas,!Nami≠Nûs,Luderitz District Hospital,10221,F2E91406-846B-7D40-9544-C522FE691720,Female,1971-04-04,Single,Active ART,None,...,2024-05-30,29.00,EF916F03-F628-4E36-9659-889E4443BCB9,Undetectable,Complete,253.0,NaN,Active ART,Yes,1
1,None,None,Unknown,99999,EEB0C17D-193C-9A46-B0F3-447E59D74CB2,Male,1960-06-06,Married,Transferred Out,None,...,2011-03-02,946.87,EF916F03-F628-4E36-9659-889E4443BCB9,LLV,Complete,252.0,NaN,Transferred Out,No,0
2,None,None,Unknown,99999,00BFC838-2A1C-AA41-9389-E480215EBE3D,Female,1969-04-14,Single,Transferred Out,None,...,NaT,NaN,NaN,NaN,NaN,251.0,NaN,Transferred Out,No,0
3,None,None,Unknown,99999,6C708B5E-FF4F-064E-9597-230BFF374297,Male,1970-01-01,Married,Transferred Out,None,...,NaT,NaN,NaN,NaN,NaN,251.0,NaN,Transferred Out,No,0
4,Ohangwena,Engela,Onamukulo Clinic,10389,792BA050-2DAF-DA48-B918-786C5B751C76,Female,1970-09-22,Single,Transferred Out,None,...,2022-01-27,19.00,EF916F03-F628-4E36-9659-889E4443BCB9,Undetectable,Complete,251.0,NaN,Transferred Out,No,0


# Add currently on TPT

In [1211]:
# Define constants
tpt_on_tx_3H = 113
tpt_on_tx_other = 240

# Update the DataFrame
fact_sentinel_event_df['currently_on_tpt'] = fact_sentinel_event_df.apply(
    lambda row: 1 if (
        row['tx_curr'] == 'Yes' and (
            # Condition 1: TPT initiated but not completed for 3HP or 3H
            (pd.notna(row['tpt_start_date']) and pd.isna(row['tpt_actual_stop_date']) and
             (last_refresh_date - row['tpt_start_date']).days < tpt_on_tx_3H and
             row['tpt_type'] in ["3HP", "3H"]) or
            # Condition 2: TPT initiated but not completed for 6H
            (pd.notna(row['tpt_start_date']) and pd.isna(row['tpt_actual_stop_date']) and
             (last_refresh_date - row['tpt_start_date']).days < tpt_on_tx_other and
             row['tpt_type'] in ["6H","6HP"]) or
            # Condition 3: Currently on TB treatment (less than 1 year since TB treatment start)
            (pd.notna(row['tbt_start_date']) and
             (last_refresh_date - row['tbt_start_date']).days < 365.25 and 
             pd.notna(row['tbt_actual_stop_date'])
        )
    ) else 0,
    axis=1
)

# Display the updated DataFrame
fact_sentinel_event_df.head()


,region,district,current_facility,facility_code,client_id,sex,date_of_birth,marital_status,patient_status,client_code,...,last_viral_load_result,last_viral_load_facility_id,last_viral_load_result_category,last_viral_load_result_status,months_since_art_start,months_since_art_restart,client_status,tx_curr,up_to_date_tpt,currently_on_tpt
0,!Karas,!Nami≠Nûs,Luderitz District Hospital,10221,F2E91406-846B-7D40-9544-C522FE691720,Female,1971-04-04,Single,Active ART,None,...,29.00,EF916F03-F628-4E36-9659-889E4443BCB9,Undetectable,Complete,253.0,NaN,Active ART,Yes,1,0
1,None,None,Unknown,99999,EEB0C17D-193C-9A46-B0F3-447E59D74CB2,Male,1960-06-06,Married,Transferred Out,None,...,946.87,EF916F03-F628-4E36-9659-889E4443BCB9,LLV,Complete,252.0,NaN,Transferred Out,No,0,0
2,None,None,Unknown,99999,00BFC838-2A1C-AA41-9389-E480215EBE3D,Female,1969-04-14,Single,Transferred Out,None,...,NaN,NaN,NaN,NaN,251.0,NaN,Transferred Out,No,0,0
3,None,None,Unknown,99999,6C708B5E-FF4F-064E-9597-230BFF374297,Male,1970-01-01,Married,Transferred Out,None,...,NaN,NaN,NaN,NaN,251.0,NaN,Transferred Out,No,0,0
4,Ohangwena,Engela,Onamukulo Clinic,10389,792BA050-2DAF-DA48-B918-786C5B751C76,Female,1970-09-22,Single,Transferred Out,None,...,19.00,EF916F03-F628-4E36-9659-889E4443BCB9,Undetectable,Complete,251.0,NaN,Transferred Out,No,0,0


# Add Incomplete or Never Initiated TPT

In [1212]:
# Define constants
tpt_on_tx_3H = 112
tpt_on_tx_other = 239

tpt_incomplete_tx_3H = 70
tpt_incomplete_tx_other = 146

# Calculate 'incomplete_or_never_initiated_tpt' column
fact_sentinel_event_df['incomplete_or_never_initiated_tpt'] = fact_sentinel_event_df.apply(
    lambda row: 1 if (
        row['tx_curr'] == 'Yes' and (
            # Condition 1: Initiation date is filled, but actual stop date is missing
            (
                pd.notna(row['tpt_start_date']) and pd.isna(row['tpt_actual_stop_date']) and (
                    # >112 days for 3HP
                    ((last_refresh_date - row['tpt_start_date']).days > tpt_on_tx_3H and
                     row['tpt_type'] in ["3HP", "3H"]) or
                    # >239 days for other regimens
                    ((last_refresh_date - row['tpt_start_date']).days > tpt_on_tx_other and
                     (row['tpt_type'] in ["6HP", "6H", "9H", "9HP"] or pd.isna(row['tpt_type'])))
                )
            ) or
            # Condition 2: Actual stop date is filled, but duration is less than required
            (
                pd.notna(row['tpt_actual_stop_date']) and pd.notna(row['tpt_start_date']) and (
                    # <70 days for 3HP
                    ((row['tpt_actual_stop_date'] - row['tpt_start_date']).days < tpt_incomplete_tx_3H and
                     row['tpt_type'] in ["3HP", "3H"]) or
                    # <146 days for other regimens
                    ((row['tpt_actual_stop_date'] - row['tpt_start_date']).days < tpt_incomplete_tx_other and
                     (row['tpt_type'] in ["6HP", "6H", "9H", "9HP"] or pd.isna(row['tpt_type'])))
                )
            ) or
            # Condition 3: No initiation date is filled, and not on TB treatment
            (
                pd.isna(row['tpt_start_date']) and (pd.isna(row['tbt_start_date'] or 
                (pd.notna(row['tbt_start_date']) and pd.notna(row['tbt_actual_stop_date']))))
            ) or 
            # Condition 4: No TPT initiation date is filled, started on TB treatment, no completion date but been on TBT for more than 1 year
            (
                pd.isna(row['tpt_start_date']) and (pd.notna(row['tbt_start_date']) and pd.isna(row['tbt_actual_stop_date'])
                and (last_refresh_date - row['tbt_start_date']).days > 365.25)
            )
        )
    ) else 0,
    axis=1
)

fact_sentinel_event_df.head()

,region,district,current_facility,facility_code,client_id,sex,date_of_birth,marital_status,patient_status,client_code,...,last_viral_load_facility_id,last_viral_load_result_category,last_viral_load_result_status,months_since_art_start,months_since_art_restart,client_status,tx_curr,up_to_date_tpt,currently_on_tpt,incomplete_or_never_initiated_tpt
0,!Karas,!Nami≠Nûs,Luderitz District Hospital,10221,F2E91406-846B-7D40-9544-C522FE691720,Female,1971-04-04,Single,Active ART,None,...,EF916F03-F628-4E36-9659-889E4443BCB9,Undetectable,Complete,253.0,NaN,Active ART,Yes,1,0,0
1,None,None,Unknown,99999,EEB0C17D-193C-9A46-B0F3-447E59D74CB2,Male,1960-06-06,Married,Transferred Out,None,...,EF916F03-F628-4E36-9659-889E4443BCB9,LLV,Complete,252.0,NaN,Transferred Out,No,0,0,0
2,None,None,Unknown,99999,00BFC838-2A1C-AA41-9389-E480215EBE3D,Female,1969-04-14,Single,Transferred Out,None,...,NaN,NaN,NaN,251.0,NaN,Transferred Out,No,0,0,0
3,None,None,Unknown,99999,6C708B5E-FF4F-064E-9597-230BFF374297,Male,1970-01-01,Married,Transferred Out,None,...,NaN,NaN,NaN,251.0,NaN,Transferred Out,No,0,0,0
4,Ohangwena,Engela,Onamukulo Clinic,10389,792BA050-2DAF-DA48-B918-786C5B751C76,Female,1970-09-22,Single,Transferred Out,None,...,EF916F03-F628-4E36-9659-889E4443BCB9,Undetectable,Complete,251.0,NaN,Transferred Out,No,0,0,0


# Compute tpt_status_two

In [1213]:
def calculate_tpt_two_status(row):
    if row['up_to_date_tpt'] == 1:
        return 'TPT completed'
    elif row['currently_on_tpt'] == 1:
        return 'Currently on TPT/TBT'
    elif row['incomplete_or_never_initiated_tpt'] == 1:
        return 'Incomplete TPT'

fact_sentinel_event_df['tpt_status_two'] = fact_sentinel_event_df.apply(calculate_tpt_two_status, axis=1)

fact_sentinel_event_df.head()

,region,district,current_facility,facility_code,client_id,sex,date_of_birth,marital_status,patient_status,client_code,...,last_viral_load_result_category,last_viral_load_result_status,months_since_art_start,months_since_art_restart,client_status,tx_curr,up_to_date_tpt,currently_on_tpt,incomplete_or_never_initiated_tpt,tpt_status_two
0,!Karas,!Nami≠Nûs,Luderitz District Hospital,10221,F2E91406-846B-7D40-9544-C522FE691720,Female,1971-04-04,Single,Active ART,None,...,Undetectable,Complete,253.0,NaN,Active ART,Yes,1,0,0,TPT completed
1,None,None,Unknown,99999,EEB0C17D-193C-9A46-B0F3-447E59D74CB2,Male,1960-06-06,Married,Transferred Out,None,...,LLV,Complete,252.0,NaN,Transferred Out,No,0,0,0,None
2,None,None,Unknown,99999,00BFC838-2A1C-AA41-9389-E480215EBE3D,Female,1969-04-14,Single,Transferred Out,None,...,NaN,NaN,251.0,NaN,Transferred Out,No,0,0,0,None
3,None,None,Unknown,99999,6C708B5E-FF4F-064E-9597-230BFF374297,Male,1970-01-01,Married,Transferred Out,None,...,NaN,NaN,251.0,NaN,Transferred Out,No,0,0,0,None
4,Ohangwena,Engela,Onamukulo Clinic,10389,792BA050-2DAF-DA48-B918-786C5B751C76,Female,1970-09-22,Single,Transferred Out,None,...,Undetectable,Complete,251.0,NaN,Transferred Out,No,0,0,0,None


# Add Retention for patients

In [1214]:
def retention_status(row):
    days_missed = (pd.to_datetime(last_refresh_date) - pd.to_datetime(row['last_next_visit_date'])).days if pd.notna(row['last_next_visit_date']) else None
    
    if row['tx_curr'] == 'Yes':
        if pd.notna(row['last_next_visit_date']) and row['last_next_visit_date'] >= last_refresh_date:
            return 'In Care'
        elif pd.isna(row['last_next_visit_date']):
            return 'In Care'
        elif pd.notna(row['last_next_visit_date']) and row['last_next_visit_date'] < last_refresh_date and 1 <= days_missed <= 28:
            return 'Missed Appointments'
    
    if pd.notna(row['last_next_visit_date']) and row['last_next_visit_date'] < last_refresh_date and 29 <= days_missed <= 90:
        return 'Treatment Interruptions'
    
    return None

fact_sentinel_event_df['retention_status'] = fact_sentinel_event_df.apply(retention_status, axis=1)

# Add patient ages at different stages cascade

In [1215]:
fact_sentinel_event_df['hiv_confirmation_age'] = (fact_sentinel_event_df['hiv_confirmation_date'] - fact_sentinel_event_df['date_of_birth']).dt.days // 365.25
fact_sentinel_event_df['hiv_enrollment_age'] = (fact_sentinel_event_df['hiv_enrollment_date'] - fact_sentinel_event_df['date_of_birth']).dt.days // 365.25
fact_sentinel_event_df['first_viral_load_result_age'] = (fact_sentinel_event_df['first_viral_load_result_date'] - fact_sentinel_event_df['date_of_birth']).dt.days // 365.25
fact_sentinel_event_df['last_viral_load_result_age'] = (fact_sentinel_event_df['last_viral_load_result_date'] - fact_sentinel_event_df['date_of_birth']).dt.days // 365.25
fact_sentinel_event_df['first_visit_age'] = (fact_sentinel_event_df['first_visit_date'] - fact_sentinel_event_df['date_of_birth']).dt.days // 365.25
fact_sentinel_event_df['last_visit_age'] = (fact_sentinel_event_df['last_visit_date'] - fact_sentinel_event_df['date_of_birth']).dt.days // 365.25
fact_sentinel_event_df['current_age'] = (last_refresh_date - fact_sentinel_event_df['date_of_birth']).dt.days // 365.25

fact_sentinel_event_df.head()

,region,district,current_facility,facility_code,client_id,sex,date_of_birth,marital_status,patient_status,client_code,...,incomplete_or_never_initiated_tpt,tpt_status_two,retention_status,hiv_confirmation_age,hiv_enrollment_age,first_viral_load_result_age,last_viral_load_result_age,first_visit_age,last_visit_age,current_age
0,!Karas,!Nami≠Nûs,Luderitz District Hospital,10221,F2E91406-846B-7D40-9544-C522FE691720,Female,1971-04-04,Single,Active ART,None,...,0,TPT completed,In Care,32.0,32.0,37.0,53.0,34.0,53.0,53.0
1,None,None,Unknown,99999,EEB0C17D-193C-9A46-B0F3-447E59D74CB2,Male,1960-06-06,Married,Transferred Out,None,...,0,None,None,40.0,43.0,48.0,50.0,46.0,51.0,64.0
2,None,None,Unknown,99999,00BFC838-2A1C-AA41-9389-E480215EBE3D,Female,1969-04-14,Single,Transferred Out,None,...,0,None,None,26.0,34.0,NaN,NaN,35.0,39.0,55.0
3,None,None,Unknown,99999,6C708B5E-FF4F-064E-9597-230BFF374297,Male,1970-01-01,Married,Transferred Out,None,...,0,None,None,32.0,33.0,NaN,NaN,36.0,38.0,54.0
4,Ohangwena,Engela,Onamukulo Clinic,10389,792BA050-2DAF-DA48-B918-786C5B751C76,Female,1970-09-22,Single,Transferred Out,None,...,0,None,None,33.0,33.0,38.0,51.0,36.0,52.0,54.0


# Add ART Duration

In [1216]:
fact_sentinel_event_df['art_duration'] = (
    (last_refresh_date.year - fact_sentinel_event_df['art_start_date'].dt.year) * 12 +
    (last_refresh_date.month - fact_sentinel_event_df['art_start_date'].dt.month)
)

fact_sentinel_event_df['art_duration'] = (last_refresh_date - fact_sentinel_event_df['art_start_date']).dt.days // 365.25

fact_sentinel_event_df.head()

,region,district,current_facility,facility_code,client_id,sex,date_of_birth,marital_status,patient_status,client_code,...,tpt_status_two,retention_status,hiv_confirmation_age,hiv_enrollment_age,first_viral_load_result_age,last_viral_load_result_age,first_visit_age,last_visit_age,current_age,art_duration
0,!Karas,!Nami≠Nûs,Luderitz District Hospital,10221,F2E91406-846B-7D40-9544-C522FE691720,Female,1971-04-04,Single,Active ART,None,...,TPT completed,In Care,32.0,32.0,37.0,53.0,34.0,53.0,53.0,21.0
1,None,None,Unknown,99999,EEB0C17D-193C-9A46-B0F3-447E59D74CB2,Male,1960-06-06,Married,Transferred Out,None,...,None,None,40.0,43.0,48.0,50.0,46.0,51.0,64.0,21.0
2,None,None,Unknown,99999,00BFC838-2A1C-AA41-9389-E480215EBE3D,Female,1969-04-14,Single,Transferred Out,None,...,None,None,26.0,34.0,NaN,NaN,35.0,39.0,55.0,20.0
3,None,None,Unknown,99999,6C708B5E-FF4F-064E-9597-230BFF374297,Male,1970-01-01,Married,Transferred Out,None,...,None,None,32.0,33.0,NaN,NaN,36.0,38.0,54.0,20.0
4,Ohangwena,Engela,Onamukulo Clinic,10389,792BA050-2DAF-DA48-B918-786C5B751C76,Female,1970-09-22,Single,Transferred Out,None,...,None,None,33.0,33.0,38.0,51.0,36.0,52.0,54.0,20.0


# Days since last visit

In [1217]:
fact_sentinel_event_df['days_since_last_visit'] = (last_refresh_date - fact_sentinel_event_df['last_visit_date']).dt.days
fact_sentinel_event_df['years_since_last_visit'] = (last_refresh_date - fact_sentinel_event_df['last_visit_date']).dt.days // 365.25

fact_sentinel_event_df.head()

,region,district,current_facility,facility_code,client_id,sex,date_of_birth,marital_status,patient_status,client_code,...,hiv_confirmation_age,hiv_enrollment_age,first_viral_load_result_age,last_viral_load_result_age,first_visit_age,last_visit_age,current_age,art_duration,days_since_last_visit,years_since_last_visit
0,!Karas,!Nami≠Nûs,Luderitz District Hospital,10221,F2E91406-846B-7D40-9544-C522FE691720,Female,1971-04-04,Single,Active ART,None,...,32.0,32.0,37.0,53.0,34.0,53.0,53.0,21.0,79.0,0.0
1,None,None,Unknown,99999,EEB0C17D-193C-9A46-B0F3-447E59D74CB2,Male,1960-06-06,Married,Transferred Out,None,...,40.0,43.0,48.0,50.0,46.0,51.0,64.0,21.0,4585.0,12.0
2,None,None,Unknown,99999,00BFC838-2A1C-AA41-9389-E480215EBE3D,Female,1969-04-14,Single,Transferred Out,None,...,26.0,34.0,NaN,NaN,35.0,39.0,55.0,20.0,5924.0,16.0
3,None,None,Unknown,99999,6C708B5E-FF4F-064E-9597-230BFF374297,Male,1970-01-01,Married,Transferred Out,None,...,32.0,33.0,NaN,NaN,36.0,38.0,54.0,20.0,5980.0,16.0
4,Ohangwena,Engela,Onamukulo Clinic,10389,792BA050-2DAF-DA48-B918-786C5B751C76,Female,1970-09-22,Single,Transferred Out,None,...,33.0,33.0,38.0,51.0,36.0,52.0,54.0,20.0,700.0,1.0


# Create dim age groups

In [1218]:
bins = [0, 4, 9, 14, 19, 24, 29, 34, 39, 44, 49, 54, 59, 64, float('inf')]
labels = ['0-4', '5-9', '10-14','15-19','20-24','25-29','30-34','35-39','40-44','45-49','50-54','55-59','60-64', '65+']

dim_age_group_df = pd.DataFrame({
    'age': range(0, 101)
})

dim_age_group_df['pepfar_age_group'] = pd.cut(dim_age_group_df['age'], bins=bins, labels=labels, right=False)

dim_age_group_df['paeds_adult_age_group'] = dim_age_group_df['age'].apply(lambda x: 'Pediatric' if x <= 19 else 'Adult')

def tri_pillar_18_classify(age):
    if 0 <= age <= 18:
        return '0-18'
    elif age == 19:
        return '19'
    elif 20 <= age <= 39:
        return '20-39'
    else:
        return '40+'

def tri_pillar_classify(age):
    if 0 <= age <= 19:
        return '0-19'
    elif 20 <= age <= 39:
        return '20-39'
    else:
        return '40+'
    
dim_age_group_df['tri_pillar_age_group_eighteen'] = dim_age_group_df['age'].apply(tri_pillar_18_classify)

dim_age_group_df['tri_pillar_age_group'] = dim_age_group_df['age'].apply(tri_pillar_classify)

dim_age_group_df.head(100)

,age,pepfar_age_group,paeds_adult_age_group,tri_pillar_age_group_eighteen,tri_pillar_age_group
0,0,0-4,Pediatric,0-18,0-19
1,1,0-4,Pediatric,0-18,0-19
2,2,0-4,Pediatric,0-18,0-19
3,3,0-4,Pediatric,0-18,0-19
4,4,5-9,Pediatric,0-18,0-19
...,...,...,...,...,...
95,95,65+,Adult,40+,40+
96,96,65+,Adult,40+,40+
97,97,65+,Adult,40+,40+
98,98,65+,Adult,40+,40+


# Add age group sort column

In [1219]:
def tri_pillar_age_group_val(age):
    if 0 <= age <= 19:
        return 1
    elif 20 <= age <= 39:
        return 2
    else:
        return 3

dim_age_group_df['tri_pillar_age_group_val'] = dim_age_group_df['age'].apply(tri_pillar_age_group_val)

dim_age_group_df['paeds_adult_age_group_val'] = dim_age_group_df['paeds_adult_age_group'].apply(
    lambda x: 1 if x == '0-18 yr old (Pediatric)' else 2
)

def pepfar_age_group_val(label):
    if label == '0-4':
        return 1
    elif label == '5-9':
        return 2
    elif label == '10-14':
        return 3
    elif label == '15-19':
        return 4
    elif label == '20-24':
        return 5
    elif label == '25-29':
        return 6
    elif label == '30-34':
        return 7
    elif label == '35-39':
        return 8
    elif label == '40-44':
        return 9
    elif label == '45-49':
        return 10
    elif label == '50-54':
        return 11
    elif label == '55-59':
        return 12
    elif label == '60-64':
        return 13
    else:  # '65+'
        return 14

dim_age_group_df['pepfar_age_group_val'] = dim_age_group_df['pepfar_age_group'].apply(pepfar_age_group_val)

dim_age_group_df.head(100)

,age,pepfar_age_group,paeds_adult_age_group,tri_pillar_age_group_eighteen,tri_pillar_age_group,tri_pillar_age_group_val,paeds_adult_age_group_val,pepfar_age_group_val
0,0,0-4,Pediatric,0-18,0-19,1,2,1
1,1,0-4,Pediatric,0-18,0-19,1,2,1
2,2,0-4,Pediatric,0-18,0-19,1,2,1
3,3,0-4,Pediatric,0-18,0-19,1,2,1
4,4,5-9,Pediatric,0-18,0-19,1,2,2
...,...,...,...,...,...,...,...,...
95,95,65+,Adult,40+,40+,3,2,14
96,96,65+,Adult,40+,40+,3,2,14
97,97,65+,Adult,40+,40+,3,2,14
98,98,65+,Adult,40+,40+,3,2,14


# Create a column that contains current age groups

In [1220]:
fact_sentinel_event_df = fact_sentinel_event_df.merge(
    dim_age_group_df[['age','pepfar_age_group', 'pepfar_age_group_val','paeds_adult_age_group', 'paeds_adult_age_group_val','tri_pillar_age_group','tri_pillar_age_group_val']], 
    left_on='current_age', 
    right_on='age', 
    how='left'
)

fact_sentinel_event_df.head()

,region,district,current_facility,facility_code,client_id,sex,date_of_birth,marital_status,patient_status,client_code,...,art_duration,days_since_last_visit,years_since_last_visit,age_y,pepfar_age_group,pepfar_age_group_val,paeds_adult_age_group,paeds_adult_age_group_val,tri_pillar_age_group,tri_pillar_age_group_val
0,!Karas,!Nami≠Nûs,Luderitz District Hospital,10221,F2E91406-846B-7D40-9544-C522FE691720,Female,1971-04-04,Single,Active ART,None,...,21.0,79.0,0.0,53,50-54,11,Adult,2,40+,3
1,None,None,Unknown,99999,EEB0C17D-193C-9A46-B0F3-447E59D74CB2,Male,1960-06-06,Married,Transferred Out,None,...,21.0,4585.0,12.0,64,65+,14,Adult,2,40+,3
2,None,None,Unknown,99999,00BFC838-2A1C-AA41-9389-E480215EBE3D,Female,1969-04-14,Single,Transferred Out,None,...,20.0,5924.0,16.0,55,55-59,12,Adult,2,40+,3
3,None,None,Unknown,99999,6C708B5E-FF4F-064E-9597-230BFF374297,Male,1970-01-01,Married,Transferred Out,None,...,20.0,5980.0,16.0,54,55-59,12,Adult,2,40+,3
4,Ohangwena,Engela,Onamukulo Clinic,10389,792BA050-2DAF-DA48-B918-786C5B751C76,Female,1970-09-22,Single,Transferred Out,None,...,20.0,700.0,1.0,54,55-59,12,Adult,2,40+,3


# Calculate the difference in months between last_refresh_date and last_viral_load_result_date

In [1221]:
def calculate_months_since_last_viral_load(date):
    return (last_refresh_date.year - date.year) * 12 + last_refresh_date.month - date.month

fact_sentinel_event_df['months_since_last_viral_load'] = fact_sentinel_event_df['last_viral_load_result_date'].apply(calculate_months_since_last_viral_load)
fact_sentinel_event_df['months_since_last_viral_load_order'] = fact_sentinel_event_df['last_viral_load_order_date'].apply(calculate_months_since_last_viral_load)

fact_sentinel_event_df['days_since_last_viral_load_order'] = (last_refresh_date - fact_sentinel_event_df['last_viral_load_order_date']).dt.days


fact_sentinel_event_df.head()

,region,district,current_facility,facility_code,client_id,sex,date_of_birth,marital_status,patient_status,client_code,...,age_y,pepfar_age_group,pepfar_age_group_val,paeds_adult_age_group,paeds_adult_age_group_val,tri_pillar_age_group,tri_pillar_age_group_val,months_since_last_viral_load,months_since_last_viral_load_order,days_since_last_viral_load_order
0,!Karas,!Nami≠Nûs,Luderitz District Hospital,10221,F2E91406-846B-7D40-9544-C522FE691720,Female,1971-04-04,Single,Active ART,None,...,53,50-54,11,Adult,2,40+,3,6.0,6.0,174.0
1,None,None,Unknown,99999,EEB0C17D-193C-9A46-B0F3-447E59D74CB2,Male,1960-06-06,Married,Transferred Out,None,...,64,65+,14,Adult,2,40+,3,164.0,164.0,5012.0
2,None,None,Unknown,99999,00BFC838-2A1C-AA41-9389-E480215EBE3D,Female,1969-04-14,Single,Transferred Out,None,...,55,55-59,12,Adult,2,40+,3,NaN,NaN,NaN
3,None,None,Unknown,99999,6C708B5E-FF4F-064E-9597-230BFF374297,Male,1970-01-01,Married,Transferred Out,None,...,54,55-59,12,Adult,2,40+,3,NaN,NaN,NaN
4,Ohangwena,Engela,Onamukulo Clinic,10389,792BA050-2DAF-DA48-B918-786C5B751C76,Female,1970-09-22,Single,Transferred Out,None,...,54,55-59,12,Adult,2,40+,3,34.0,34.0,1028.0


# Add VL Eligibility

In [1222]:
def classify_vl_eligibility(row):
    # Monitored per Guidelines
    if row['tx_curr'] == 'Yes':
        if row['months_since_art_start'] < 6 or row['months_since_art_restart'] < 6 :
            return 'Monitored per Guidelines'

        if row['last_viral_load_result'] >= 40 and row['months_since_last_viral_load'] <= 3:
            return 'Monitored per Guidelines'

        if (row['last_breast_feeding'] == 'Yes' or row['last_pregnancy_status'] == 'Yes') and row['months_since_last_viral_load'] <= 3:
            return 'Monitored per Guidelines'

        if row['current_age'] > 19 and row['last_breast_feeding'] != 'Yes' and row['last_pregnancy_status'] != 'Yes' and row['last_viral_load_result'] < 40 and row['months_since_last_viral_load'] <= 12:
            return 'Monitored per Guidelines'

        if row['current_age'] <= 19 and row['last_breast_feeding'] != 'Yes' and row['last_pregnancy_status'] != 'Yes' and row['last_viral_load_result'] < 40 and row['months_since_last_viral_load'] <= 6:
            return 'Monitored per Guidelines'

    # Slightly Delayed
    if row['tx_curr'] == 'Yes':
        if row['last_viral_load_result'] >= 40 and 3 < row['months_since_last_viral_load'] <= 4:
            return 'Slightly Delayed'

        if (row['last_breast_feeding'] == 'Yes' or row['last_pregnancy_status'] == 'Yes') and 3 < row['months_since_last_viral_load'] <= 4:
            return 'Slightly Delayed'

        if row['current_age'] > 19 and row['last_breast_feeding'] != 'Yes' and row['last_pregnancy_status'] != 'Yes' and row['last_viral_load_result'] < 40 and 12 < row['months_since_last_viral_load'] <= 14:
            return 'Slightly Delayed'

        if row['current_age'] <= 19 and row['last_breast_feeding'] != 'Yes' and row['last_pregnancy_status'] != 'Yes' and row['last_viral_load_result'] < 40 and 6 < row['months_since_last_viral_load'] <= 8:
            return 'Slightly Delayed'
                
        # add patients with an order date and no result and the test was done in the last 14 days
        if row['days_since_last_viral_load_order'] <= 14 and pd.isna(row['last_viral_load_result']):
            return 'Slightly Delayed'

    # Delayed
    if row['tx_curr'] == 'Yes':
        if row['last_viral_load_result'] >= 40 and row['months_since_last_viral_load'] > 4:
            return 'Delayed'

        if (row['last_breast_feeding'] == 'Yes' or row['last_pregnancy_status'] == 'Yes') and row['months_since_last_viral_load'] > 4:
            return 'Delayed'

        if row['current_age'] > 19 and row['last_breast_feeding'] != 'Yes' and row['last_pregnancy_status'] != 'Yes' and row['last_viral_load_result'] < 40 and row['months_since_last_viral_load'] > 14:
            return 'Delayed'

        if row['current_age'] <= 19 and row['last_breast_feeding'] != 'Yes' and row['last_pregnancy_status'] != 'Yes' and row['last_viral_load_result'] < 40 and row['months_since_last_viral_load'] > 8:
            return 'Delayed'

    # Default to Delayed if TxCurr is 'Yes' and no other conditions are met
    if row['tx_curr'] == 'Yes':
        return 'Delayed'

    # If none of the conditions are met, return 'Not Classified'
    return None


fact_sentinel_event_df['vl_eligibility'] = fact_sentinel_event_df.apply(lambda row: classify_vl_eligibility(row), axis=1)

fact_sentinel_event_df.head()

,region,district,current_facility,facility_code,client_id,sex,date_of_birth,marital_status,patient_status,client_code,...,pepfar_age_group,pepfar_age_group_val,paeds_adult_age_group,paeds_adult_age_group_val,tri_pillar_age_group,tri_pillar_age_group_val,months_since_last_viral_load,months_since_last_viral_load_order,days_since_last_viral_load_order,vl_eligibility
0,!Karas,!Nami≠Nûs,Luderitz District Hospital,10221,F2E91406-846B-7D40-9544-C522FE691720,Female,1971-04-04,Single,Active ART,None,...,50-54,11,Adult,2,40+,3,6.0,6.0,174.0,Monitored per Guidelines
1,None,None,Unknown,99999,EEB0C17D-193C-9A46-B0F3-447E59D74CB2,Male,1960-06-06,Married,Transferred Out,None,...,65+,14,Adult,2,40+,3,164.0,164.0,5012.0,None
2,None,None,Unknown,99999,00BFC838-2A1C-AA41-9389-E480215EBE3D,Female,1969-04-14,Single,Transferred Out,None,...,55-59,12,Adult,2,40+,3,NaN,NaN,NaN,None
3,None,None,Unknown,99999,6C708B5E-FF4F-064E-9597-230BFF374297,Male,1970-01-01,Married,Transferred Out,None,...,55-59,12,Adult,2,40+,3,NaN,NaN,NaN,None
4,Ohangwena,Engela,Onamukulo Clinic,10389,792BA050-2DAF-DA48-B918-786C5B751C76,Female,1970-09-22,Single,Transferred Out,None,...,55-59,12,Adult,2,40+,3,34.0,34.0,1028.0,None


# Add reason for next VL

In [1223]:
def categorize_vl_reason(row):
    if row['months_since_art_start'] is not None and row['months_since_art_start'] > 6 and pd.notnull(row['art_start_date']) and pd.isnull(row['first_viral_load_result_date']):
        return 'First Test'
    elif row['months_since_last_viral_load'] > 3 and row['last_pregnancy_status'] == 'Yes':
        return 'Pregnant Women'
    elif row['months_since_last_viral_load'] > 3 and row['last_breast_feeding'] == 'Yes':
        return 'BF Women'
    elif row['months_since_last_viral_load'] > 3 and row['last_viral_load_result_category'] == 'LLV':
        return 'Low Level Viraemia'
    elif row['months_since_last_viral_load'] > 6 and row['current_age'] <= 19:
        return 'Children And Adolescents Under 19'
    elif row['months_since_last_viral_load'] > 12 and row['current_age'] > 19:
        return 'Treatment Monitoring For Adults'
    else:
        return None

fact_sentinel_event_df['reason_for_next_vl'] = fact_sentinel_event_df.apply(categorize_vl_reason, axis=1)

fact_sentinel_event_df.head()

,region,district,current_facility,facility_code,client_id,sex,date_of_birth,marital_status,patient_status,client_code,...,pepfar_age_group_val,paeds_adult_age_group,paeds_adult_age_group_val,tri_pillar_age_group,tri_pillar_age_group_val,months_since_last_viral_load,months_since_last_viral_load_order,days_since_last_viral_load_order,vl_eligibility,reason_for_next_vl
0,!Karas,!Nami≠Nûs,Luderitz District Hospital,10221,F2E91406-846B-7D40-9544-C522FE691720,Female,1971-04-04,Single,Active ART,None,...,11,Adult,2,40+,3,6.0,6.0,174.0,Monitored per Guidelines,None
1,None,None,Unknown,99999,EEB0C17D-193C-9A46-B0F3-447E59D74CB2,Male,1960-06-06,Married,Transferred Out,None,...,14,Adult,2,40+,3,164.0,164.0,5012.0,None,Low Level Viraemia
2,None,None,Unknown,99999,00BFC838-2A1C-AA41-9389-E480215EBE3D,Female,1969-04-14,Single,Transferred Out,None,...,12,Adult,2,40+,3,NaN,NaN,NaN,None,First Test
3,None,None,Unknown,99999,6C708B5E-FF4F-064E-9597-230BFF374297,Male,1970-01-01,Married,Transferred Out,None,...,12,Adult,2,40+,3,NaN,NaN,NaN,None,First Test
4,Ohangwena,Engela,Onamukulo Clinic,10389,792BA050-2DAF-DA48-B918-786C5B751C76,Female,1970-09-22,Single,Transferred Out,None,...,12,Adult,2,40+,3,34.0,34.0,1028.0,None,Treatment Monitoring For Adults


# Add next VL date

In [1224]:
def calculate_next_vl_date(row):
    if pd.isnull(row['last_viral_load_result_date']):
        return None
    elif row['reason_for_next_vl'] == 'First Test':
        return row['last_viral_load_result_date'] + pd.DateOffset(months=6)
    elif row['reason_for_next_vl'] == 'Pregnant Women':
        return row['last_viral_load_result_date'] + pd.DateOffset(months=3)
    elif row['reason_for_next_vl'] == 'BF Women':
        return row['last_viral_load_result_date'] + pd.DateOffset(months=3)
    elif row['reason_for_next_vl'] == 'Low Level Viraemia':
        return row['last_viral_load_result_date'] + pd.DateOffset(months=6)
    elif row['reason_for_next_vl'] == 'Children And Adolescents Under 19':
        return row['last_viral_load_result_date'] + pd.DateOffset(months=6)
    elif row['reason_for_next_vl'] == 'Treatment Monitoring For Adults':
        return row['last_viral_load_result_date'] + pd.DateOffset(months=12)
    else:
        return None

fact_sentinel_event_df['next_vl_date'] = fact_sentinel_event_df.apply(calculate_next_vl_date, axis=1)


# Compute if patient has ever been initiated on ART  

In [1225]:
def determine_has_been_initiated_on_art(row):
    return 'Yes' if pd.notna(row['art_start_date']) else 'No'
    
fact_sentinel_event_df['has_ever_been_initiated_on_art'] = fact_sentinel_event_df.apply(lambda row: determine_has_been_initiated_on_art(row), axis=1)

fact_sentinel_event_df.head()

,region,district,current_facility,facility_code,client_id,sex,date_of_birth,marital_status,patient_status,client_code,...,paeds_adult_age_group_val,tri_pillar_age_group,tri_pillar_age_group_val,months_since_last_viral_load,months_since_last_viral_load_order,days_since_last_viral_load_order,vl_eligibility,reason_for_next_vl,next_vl_date,has_ever_been_initiated_on_art
0,!Karas,!Nami≠Nûs,Luderitz District Hospital,10221,F2E91406-846B-7D40-9544-C522FE691720,Female,1971-04-04,Single,Active ART,None,...,2,40+,3,6.0,6.0,174.0,Monitored per Guidelines,None,NaT,Yes
1,None,None,Unknown,99999,EEB0C17D-193C-9A46-B0F3-447E59D74CB2,Male,1960-06-06,Married,Transferred Out,None,...,2,40+,3,164.0,164.0,5012.0,None,Low Level Viraemia,2011-09-02,Yes
2,None,None,Unknown,99999,00BFC838-2A1C-AA41-9389-E480215EBE3D,Female,1969-04-14,Single,Transferred Out,None,...,2,40+,3,NaN,NaN,NaN,None,First Test,NaT,Yes
3,None,None,Unknown,99999,6C708B5E-FF4F-064E-9597-230BFF374297,Male,1970-01-01,Married,Transferred Out,None,...,2,40+,3,NaN,NaN,NaN,None,First Test,NaT,Yes
4,Ohangwena,Engela,Onamukulo Clinic,10389,792BA050-2DAF-DA48-B918-786C5B751C76,Female,1970-09-22,Single,Transferred Out,None,...,2,40+,3,34.0,34.0,1028.0,None,Treatment Monitoring For Adults,2023-01-27,Yes


# Check if last VL is valid

In [1226]:
def is_last_viral_load_valid(row, df):
    row = df.iloc[row.name]
    if (row['tx_curr'] == 'Yes' and
        row['last_PBFW_status'] == 'Yes' and
        pd.notna(row['last_viral_load_result']) and
        pd.notna(row['art_start_date']) and
        (last_refresh_date - pd.to_datetime(row['art_start_date'])).days >= 90 and
        row['vl_eligibility'] == 'Monitored per Guidelines'):
        return 'Yes'

    elif (row['tx_curr'] == 'Yes' and
          row['last_PBFW_status'] == 'Yes' and
          pd.notna(row['last_viral_load_result']) and
          pd.isna(row['art_start_date']) and
          pd.notna(row['hiv_confirmation_date']) and
          (last_refresh_date - pd.to_datetime(row['hiv_confirmation_date'])).days >= 90 and
          row['vl_eligibility'] == 'Monitored per Guidelines'):
        return 'Yes'

    elif (row['tx_curr'] == 'Yes' and
          (row['last_PBFW_status'] == 'No' or pd.isna(row['last_PBFW_status'])) and
          row['current_age'] >= 20 and
          pd.notna(row['last_viral_load_result']) and
          pd.notna(row['art_start_date']) and
          (last_refresh_date - pd.to_datetime(row['art_start_date'])).days >= 182 and
          row['vl_eligibility'] == 'Monitored per Guidelines'):
        return 'Yes'

    elif (row['tx_curr'] == 'Yes' and
          (row['last_PBFW_status'] == 'No' or pd.isna(row['last_PBFW_status'])) and
          row['current_age'] >= 20 and
          pd.notna(row['last_viral_load_result']) and
          pd.isna(row['art_start_date']) and
          pd.notna(row['hiv_confirmation_date']) and
          (last_refresh_date - pd.to_datetime(row['hiv_confirmation_date'])).days >= 182 and
          row['vl_eligibility'] == 'Monitored per Guidelines'):
        return 'Yes'

    elif (row['tx_curr'] == 'Yes' and
          (row['last_PBFW_status'] == 'No' or pd.isna(row['last_PBFW_status'])) and
          row['current_age'] <= 19 and
          pd.notna(row['last_viral_load_result']) and
          pd.notna(row['art_start_date']) and
          (last_refresh_date - pd.to_datetime(row['art_start_date'])).days >= 182 and
          row['vl_eligibility'] == 'Monitored per Guidelines'):
        return 'Yes'

    elif (row['tx_curr'] == 'Yes' and
          (row['last_PBFW_status'] == 'No' or pd.isna(row['last_PBFW_status'])) and
          row['current_age'] <= 19 and
          pd.notna(row['last_viral_load_result']) and
          pd.isna(row['art_start_date']) and
          pd.notna(row['hiv_confirmation_date']) and
          (last_refresh_date - pd.to_datetime(row['hiv_confirmation_date'])).days >= 182 and
          row['vl_eligibility'] == 'Monitored per Guidelines'):
        return 'Yes'

    else:
        return 'No'

    
fact_sentinel_event_df['is_last_viral_load_valid'] = fact_sentinel_event_df.apply(is_last_viral_load_valid, axis=1, df=fact_sentinel_event_df)

fact_sentinel_event_df.head()

,region,district,current_facility,facility_code,client_id,sex,date_of_birth,marital_status,patient_status,client_code,...,tri_pillar_age_group,tri_pillar_age_group_val,months_since_last_viral_load,months_since_last_viral_load_order,days_since_last_viral_load_order,vl_eligibility,reason_for_next_vl,next_vl_date,has_ever_been_initiated_on_art,is_last_viral_load_valid
0,!Karas,!Nami≠Nûs,Luderitz District Hospital,10221,F2E91406-846B-7D40-9544-C522FE691720,Female,1971-04-04,Single,Active ART,None,...,40+,3,6.0,6.0,174.0,Monitored per Guidelines,None,NaT,Yes,Yes
1,None,None,Unknown,99999,EEB0C17D-193C-9A46-B0F3-447E59D74CB2,Male,1960-06-06,Married,Transferred Out,None,...,40+,3,164.0,164.0,5012.0,None,Low Level Viraemia,2011-09-02,Yes,No
2,None,None,Unknown,99999,00BFC838-2A1C-AA41-9389-E480215EBE3D,Female,1969-04-14,Single,Transferred Out,None,...,40+,3,NaN,NaN,NaN,None,First Test,NaT,Yes,No
3,None,None,Unknown,99999,6C708B5E-FF4F-064E-9597-230BFF374297,Male,1970-01-01,Married,Transferred Out,None,...,40+,3,NaN,NaN,NaN,None,First Test,NaT,Yes,No
4,Ohangwena,Engela,Onamukulo Clinic,10389,792BA050-2DAF-DA48-B918-786C5B751C76,Female,1970-09-22,Single,Transferred Out,None,...,40+,3,34.0,34.0,1028.0,None,Treatment Monitoring For Adults,2023-01-27,Yes,No


# Check if patient is virally suppressed

In [1227]:
def determine_is_virally_suppressed(row, df):
    row = df.iloc[row.name]
    if row['is_last_viral_load_valid'] == 'Yes' and row['last_viral_load_result_category'] == 'Undetectable':
        return 'Yes'
    else:
        return None
    
fact_sentinel_event_df['is_virally_suppressed'] = fact_sentinel_event_df.apply(determine_is_virally_suppressed, axis=1, df=fact_sentinel_event_df)

fact_sentinel_event_df.head()

,region,district,current_facility,facility_code,client_id,sex,date_of_birth,marital_status,patient_status,client_code,...,tri_pillar_age_group_val,months_since_last_viral_load,months_since_last_viral_load_order,days_since_last_viral_load_order,vl_eligibility,reason_for_next_vl,next_vl_date,has_ever_been_initiated_on_art,is_last_viral_load_valid,is_virally_suppressed
0,!Karas,!Nami≠Nûs,Luderitz District Hospital,10221,F2E91406-846B-7D40-9544-C522FE691720,Female,1971-04-04,Single,Active ART,None,...,3,6.0,6.0,174.0,Monitored per Guidelines,None,NaT,Yes,Yes,Yes
1,None,None,Unknown,99999,EEB0C17D-193C-9A46-B0F3-447E59D74CB2,Male,1960-06-06,Married,Transferred Out,None,...,3,164.0,164.0,5012.0,None,Low Level Viraemia,2011-09-02,Yes,No,None
2,None,None,Unknown,99999,00BFC838-2A1C-AA41-9389-E480215EBE3D,Female,1969-04-14,Single,Transferred Out,None,...,3,NaN,NaN,NaN,None,First Test,NaT,Yes,No,None
3,None,None,Unknown,99999,6C708B5E-FF4F-064E-9597-230BFF374297,Male,1970-01-01,Married,Transferred Out,None,...,3,NaN,NaN,NaN,None,First Test,NaT,Yes,No,None
4,Ohangwena,Engela,Onamukulo Clinic,10389,792BA050-2DAF-DA48-B918-786C5B751C76,Female,1970-09-22,Single,Transferred Out,None,...,3,34.0,34.0,1028.0,None,Treatment Monitoring For Adults,2023-01-27,Yes,No,None


# Check if patient is not virally suppressed

In [1228]:
def determine_is_not_virally_suppressed(row, df):
    row = df.iloc[row.name]
    if row['is_last_viral_load_valid'] == 'Yes' and row['last_viral_load_result_category'] == 'Unsuppressed':
        return 'Yes'
    else:
        return None
    
fact_sentinel_event_df['is_not_virally_suppressed'] = fact_sentinel_event_df.apply(determine_is_not_virally_suppressed, axis=1, df=fact_sentinel_event_df)

fact_sentinel_event_df.head()

,region,district,current_facility,facility_code,client_id,sex,date_of_birth,marital_status,patient_status,client_code,...,months_since_last_viral_load,months_since_last_viral_load_order,days_since_last_viral_load_order,vl_eligibility,reason_for_next_vl,next_vl_date,has_ever_been_initiated_on_art,is_last_viral_load_valid,is_virally_suppressed,is_not_virally_suppressed
0,!Karas,!Nami≠Nûs,Luderitz District Hospital,10221,F2E91406-846B-7D40-9544-C522FE691720,Female,1971-04-04,Single,Active ART,None,...,6.0,6.0,174.0,Monitored per Guidelines,None,NaT,Yes,Yes,Yes,None
1,None,None,Unknown,99999,EEB0C17D-193C-9A46-B0F3-447E59D74CB2,Male,1960-06-06,Married,Transferred Out,None,...,164.0,164.0,5012.0,None,Low Level Viraemia,2011-09-02,Yes,No,None,None
2,None,None,Unknown,99999,00BFC838-2A1C-AA41-9389-E480215EBE3D,Female,1969-04-14,Single,Transferred Out,None,...,NaN,NaN,NaN,None,First Test,NaT,Yes,No,None,None
3,None,None,Unknown,99999,6C708B5E-FF4F-064E-9597-230BFF374297,Male,1970-01-01,Married,Transferred Out,None,...,NaN,NaN,NaN,None,First Test,NaT,Yes,No,None,None
4,Ohangwena,Engela,Onamukulo Clinic,10389,792BA050-2DAF-DA48-B918-786C5B751C76,Female,1970-09-22,Single,Transferred Out,None,...,34.0,34.0,1028.0,None,Treatment Monitoring For Adults,2023-01-27,Yes,No,None,None


# Compute needs VL test

In [1229]:
def determine_needs_VL_test(row, df):
    row = df.iloc[row.name]
    if (row['months_since_art_start'] > 6 and pd.notna(row['first_viral_load_result_date']) and row['last_breast_feeding'] != 'Yes') and (row['is_not_virally_suppressed'] != 'Yes' and row['is_virally_suppressed'] != 'Yes'):
        return 'Yes'
    elif (row['last_viral_load_result_category'] == 'Undetectable' and row['last_next_visit_date'] >= last_refresh_date and row['months_since_last_viral_load'] > 12 and row['last_breast_feeding'] != 'Yes') and (row['is_not_virally_suppressed'] != 'Yes' and row['is_virally_suppressed'] != 'Yes'):
        return 'Yes'
    elif (row['last_viral_load_result_category'] == 'Unsuppressed' and row['last_next_visit_date'] >= last_refresh_date and row['months_since_last_viral_load'] > 3 and row['last_breast_feeding'] != 'Yes') and (row['is_not_virally_suppressed'] != 'Yes' and row['is_virally_suppressed'] != 'Yes'):
        return 'Yes'
    elif (row['months_since_art_start'] > 3 and pd.notna(row['first_viral_load_result_date']) and row['last_breast_feeding'] == 'Yes') and (row['is_not_virally_suppressed'] != 'Yes' and row['is_virally_suppressed'] != 'Yes'):
        return 'Yes'
    elif (row['last_viral_load_result_category'] == 'Undetectable' and row['last_next_visit_date'] >= last_refresh_date and row['months_since_last_viral_load'] > 6 and row['last_breast_feeding'] == 'Yes') and (row['is_not_virally_suppressed'] != 'Yes' and row['is_virally_suppressed'] != 'Yes'):
        return 'Yes'
    elif (row['last_viral_load_result_category'] == 'Unsuppressed' and row['last_next_visit_date'] >= last_refresh_date and row['months_since_last_viral_load'] > 3 and row['last_breast_feeding'] == 'Yes') and (row['is_not_virally_suppressed'] != 'Yes' and row['is_virally_suppressed'] != 'Yes'):
        return 'Yes'
    else:
        return None
    
fact_sentinel_event_df['needs_vl_test'] = fact_sentinel_event_df.apply(determine_needs_VL_test, axis=1, df=fact_sentinel_event_df)

fact_sentinel_event_df.head()

,region,district,current_facility,facility_code,client_id,sex,date_of_birth,marital_status,patient_status,client_code,...,months_since_last_viral_load_order,days_since_last_viral_load_order,vl_eligibility,reason_for_next_vl,next_vl_date,has_ever_been_initiated_on_art,is_last_viral_load_valid,is_virally_suppressed,is_not_virally_suppressed,needs_vl_test
0,!Karas,!Nami≠Nûs,Luderitz District Hospital,10221,F2E91406-846B-7D40-9544-C522FE691720,Female,1971-04-04,Single,Active ART,None,...,6.0,174.0,Monitored per Guidelines,None,NaT,Yes,Yes,Yes,None,None
1,None,None,Unknown,99999,EEB0C17D-193C-9A46-B0F3-447E59D74CB2,Male,1960-06-06,Married,Transferred Out,None,...,164.0,5012.0,None,Low Level Viraemia,2011-09-02,Yes,No,None,None,Yes
2,None,None,Unknown,99999,00BFC838-2A1C-AA41-9389-E480215EBE3D,Female,1969-04-14,Single,Transferred Out,None,...,NaN,NaN,None,First Test,NaT,Yes,No,None,None,None
3,None,None,Unknown,99999,6C708B5E-FF4F-064E-9597-230BFF374297,Male,1970-01-01,Married,Transferred Out,None,...,NaN,NaN,None,First Test,NaT,Yes,No,None,None,None
4,Ohangwena,Engela,Onamukulo Clinic,10389,792BA050-2DAF-DA48-B918-786C5B751C76,Female,1970-09-22,Single,Transferred Out,None,...,34.0,1028.0,None,Treatment Monitoring For Adults,2023-01-27,Yes,No,None,None,Yes


# Assign MMP Status

In [1230]:
def assign_mmp(row,df):
    row = df.iloc[row.name]
    if (row['tx_curr'] == 'Yes' and 
        pd.notna(row['last_next_visit_date']) and 
        pd.notna(row['last_visit_date']) and
        0 <= (pd.to_datetime(row['last_next_visit_date']) - pd.to_datetime(row['last_visit_date'])).days < 84):
        return '< 3 MMP'
    elif (row['tx_curr'] == 'Yes' and
          pd.notna(row['last_next_visit_date']) and 
          pd.notna(row['last_visit_date']) and
          84 <= (pd.to_datetime(row['last_next_visit_date']) - pd.to_datetime(row['last_visit_date'])).days < 168):
        return '3-5 MMP'
    elif (row['tx_curr'] == 'Yes' and
          pd.notna(row['last_next_visit_date']) and
          pd.notna(row['last_visit_date']) and
          (pd.to_datetime(row['last_next_visit_date']) - pd.to_datetime(row['last_visit_date'])).days >= 168):
        return '6+ MMP'
    elif (row['tx_curr'] == 'Yes' and
          pd.isna(row['last_next_visit_date'])):
        return '6+ MMP'
    else:
        return None

fact_sentinel_event_df['mmp_status'] = fact_sentinel_event_df.apply(assign_mmp, axis=1, df=fact_sentinel_event_df)

fact_sentinel_event_df.head()


,region,district,current_facility,facility_code,client_id,sex,date_of_birth,marital_status,patient_status,client_code,...,days_since_last_viral_load_order,vl_eligibility,reason_for_next_vl,next_vl_date,has_ever_been_initiated_on_art,is_last_viral_load_valid,is_virally_suppressed,is_not_virally_suppressed,needs_vl_test,mmp_status
0,!Karas,!Nami≠Nûs,Luderitz District Hospital,10221,F2E91406-846B-7D40-9544-C522FE691720,Female,1971-04-04,Single,Active ART,None,...,174.0,Monitored per Guidelines,None,NaT,Yes,Yes,Yes,None,None,6+ MMP
1,None,None,Unknown,99999,EEB0C17D-193C-9A46-B0F3-447E59D74CB2,Male,1960-06-06,Married,Transferred Out,None,...,5012.0,None,Low Level Viraemia,2011-09-02,Yes,No,None,None,Yes,None
2,None,None,Unknown,99999,00BFC838-2A1C-AA41-9389-E480215EBE3D,Female,1969-04-14,Single,Transferred Out,None,...,NaN,None,First Test,NaT,Yes,No,None,None,None,None
3,None,None,Unknown,99999,6C708B5E-FF4F-064E-9597-230BFF374297,Male,1970-01-01,Married,Transferred Out,None,...,NaN,None,First Test,NaT,Yes,No,None,None,None,None
4,Ohangwena,Engela,Onamukulo Clinic,10389,792BA050-2DAF-DA48-B918-786C5B751C76,Female,1970-09-22,Single,Transferred Out,None,...,1028.0,None,Treatment Monitoring For Adults,2023-01-27,Yes,No,None,None,Yes,None


# Add Six MMP Eligibility

In [1231]:
def determine_six_mmp_eligibility(row, df):
    row = df.iloc[row.name]
    if (row['current_age'] > 2 and (pd.isnull(row['tbt_start_date']) or row['tbt_duration'] >= 6) and (row['is_last_viral_load_valid'] == 'Yes' and row['last_viral_load_result'] < 40) and row['months_since_art_start'] >= 6  and (row['last_regimen_line'] == '1' or row['last_regimen_line'] == '2') and pd.isnull(row['last_oi_other'])):
        return 'Yes'
    else:
        return None
        
max_last_visit_month = last_refresh_date.month
max_last_visit_year = last_refresh_date.year

if max_last_visit_month == 1:  # Handle the edge case for January
    prev_last_visit_month = 12
    prev_last_visit_year = max_last_visit_year - 1
else:
    prev_last_visit_month = max_last_visit_month - 1
    prev_last_visit_year = max_last_visit_year

fact_sentinel_event_df['last_visit_month'] = fact_sentinel_event_df['last_visit_date'].dt.month
fact_sentinel_event_df['last_visit_year'] = fact_sentinel_event_df['last_visit_date'].dt.year

fact_sentinel_event_df['mmp_status_current'] = fact_sentinel_event_df.apply(
    lambda row: row['mmp_status'] if (row['last_visit_month'] == prev_last_visit_month and row['last_visit_year'] == prev_last_visit_year) else None,
    axis=1
)

fact_sentinel_event_df['mmp_status_adult'] = fact_sentinel_event_df['paeds_adult_age_group'].apply(lambda x: 'Yes' if 'adult' in str(x).lower() else None)

fact_sentinel_event_df['mmp_status_paed'] = fact_sentinel_event_df['paeds_adult_age_group'].apply(lambda x: 'Yes' if 'pediatric' in str(x).lower() else None)

fact_sentinel_event_df['six_mmp_eligible'] = fact_sentinel_event_df.apply(determine_six_mmp_eligibility, axis=1, df=fact_sentinel_event_df)

fact_sentinel_event_df.head()

,region,district,current_facility,facility_code,client_id,sex,date_of_birth,marital_status,patient_status,client_code,...,is_virally_suppressed,is_not_virally_suppressed,needs_vl_test,mmp_status,last_visit_month,last_visit_year,mmp_status_current,mmp_status_adult,mmp_status_paed,six_mmp_eligible
0,!Karas,!Nami≠Nûs,Luderitz District Hospital,10221,F2E91406-846B-7D40-9544-C522FE691720,Female,1971-04-04,Single,Active ART,None,...,Yes,None,None,6+ MMP,9.0,2024.0,None,Yes,None,Yes
1,None,None,Unknown,99999,EEB0C17D-193C-9A46-B0F3-447E59D74CB2,Male,1960-06-06,Married,Transferred Out,None,...,None,None,Yes,None,5.0,2012.0,None,Yes,None,None
2,None,None,Unknown,99999,00BFC838-2A1C-AA41-9389-E480215EBE3D,Female,1969-04-14,Single,Transferred Out,None,...,None,None,None,None,9.0,2008.0,None,Yes,None,None
3,None,None,Unknown,99999,6C708B5E-FF4F-064E-9597-230BFF374297,Male,1970-01-01,Married,Transferred Out,None,...,None,None,None,None,7.0,2008.0,None,Yes,None,None
4,Ohangwena,Engela,Onamukulo Clinic,10389,792BA050-2DAF-DA48-B918-786C5B751C76,Female,1970-09-22,Single,Transferred Out,None,...,None,None,Yes,None,12.0,2022.0,None,Yes,None,None


# Add Six MMP Eligibility But Not Given

In [1232]:
fact_sentinel_event_df['six_mmp_eligible_but_not_given'] = fact_sentinel_event_df.apply(
    lambda row: 'Yes' if row['six_mmp_eligible'] == 'Yes' and row['mmp_status'] != '6+ MMP' else 'No', 
    axis=1
)

fact_sentinel_event_df.head()

,region,district,current_facility,facility_code,client_id,sex,date_of_birth,marital_status,patient_status,client_code,...,is_not_virally_suppressed,needs_vl_test,mmp_status,last_visit_month,last_visit_year,mmp_status_current,mmp_status_adult,mmp_status_paed,six_mmp_eligible,six_mmp_eligible_but_not_given
0,!Karas,!Nami≠Nûs,Luderitz District Hospital,10221,F2E91406-846B-7D40-9544-C522FE691720,Female,1971-04-04,Single,Active ART,None,...,None,None,6+ MMP,9.0,2024.0,None,Yes,None,Yes,No
1,None,None,Unknown,99999,EEB0C17D-193C-9A46-B0F3-447E59D74CB2,Male,1960-06-06,Married,Transferred Out,None,...,None,Yes,None,5.0,2012.0,None,Yes,None,None,No
2,None,None,Unknown,99999,00BFC838-2A1C-AA41-9389-E480215EBE3D,Female,1969-04-14,Single,Transferred Out,None,...,None,None,None,9.0,2008.0,None,Yes,None,None,No
3,None,None,Unknown,99999,6C708B5E-FF4F-064E-9597-230BFF374297,Male,1970-01-01,Married,Transferred Out,None,...,None,None,None,7.0,2008.0,None,Yes,None,None,No
4,Ohangwena,Engela,Onamukulo Clinic,10389,792BA050-2DAF-DA48-B918-786C5B751C76,Female,1970-09-22,Single,Transferred Out,None,...,None,Yes,None,12.0,2022.0,None,Yes,None,None,No


# Compute currrent cascade status

In [1233]:
def determine_art_outcomes(row):
    if row['is_virally_suppressed'] == 'Yes':
        return 'VL Suppressed'
    elif row['is_not_virally_suppressed'] == 'Yes':
        return 'VL Not Suppressed'
    elif row['needs_vl_test'] == 'Yes':
        return 'Needs VL Test'
    elif row['months_since_art_start'] < 6:
        return 'Recently initiated'
    elif row['has_ever_been_initiated_on_art'] == 'No':
        return 'Not initiated on ART'
    elif row['years_since_last_visit'] > 2:
        return 'Case Closed'
    elif row['patient_status'] == 'Deceased':
        return 'Died'
    else:
        return 'Unknown'

fact_sentinel_event_df['art_outcomes'] = fact_sentinel_event_df.apply(determine_art_outcomes, axis=1)

fact_sentinel_event_df.head()

environment.log_message(f'Finished adding computed columns')

[2024-11-20T18:01:12.391111] QE Data Task Running - INFO: Finished adding computed columns
QE Data Task Running >>>: Finished adding computed columns


# Change dates from datetime to date 

In [1234]:
fact_sentinel_event_df = fact_sentinel_event_df.apply(lambda col: col.dt.date if col.dtype == 'datetime64[ns]' else col)

fact_sentinel_event_df.head()

,region,district,current_facility,facility_code,client_id,sex,date_of_birth,marital_status,patient_status,client_code,...,needs_vl_test,mmp_status,last_visit_month,last_visit_year,mmp_status_current,mmp_status_adult,mmp_status_paed,six_mmp_eligible,six_mmp_eligible_but_not_given,art_outcomes
0,!Karas,!Nami≠Nûs,Luderitz District Hospital,10221,F2E91406-846B-7D40-9544-C522FE691720,Female,1971-04-04,Single,Active ART,None,...,None,6+ MMP,9.0,2024.0,None,Yes,None,Yes,No,VL Suppressed
1,None,None,Unknown,99999,EEB0C17D-193C-9A46-B0F3-447E59D74CB2,Male,1960-06-06,Married,Transferred Out,None,...,Yes,None,5.0,2012.0,None,Yes,None,None,No,Needs VL Test
2,None,None,Unknown,99999,00BFC838-2A1C-AA41-9389-E480215EBE3D,Female,1969-04-14,Single,Transferred Out,None,...,None,None,9.0,2008.0,None,Yes,None,None,No,Case Closed
3,None,None,Unknown,99999,6C708B5E-FF4F-064E-9597-230BFF374297,Male,1970-01-01,Married,Transferred Out,None,...,None,None,7.0,2008.0,None,Yes,None,None,No,Case Closed
4,Ohangwena,Engela,Onamukulo Clinic,10389,792BA050-2DAF-DA48-B918-786C5B751C76,Female,1970-09-22,Single,Transferred Out,None,...,Yes,None,12.0,2022.0,None,Yes,None,None,No,Needs VL Test


# Reorder the columns in fact_sentinel_event_df

In [1235]:
new_column_order = [
    'client_id', 'region', 'district', 'facility_code', 'current_facility', 'id_facility_current',  'transfer_status', 'transfer_date',
    'quantum_number', 'art_number', 'art_number_legacy', 'client_code', 'pharmacy_code', 
    'pmtct_number',  'first_name', 'last_name', 'contact_number', 'alt_contact_number', 'date_of_birth', 'sex', 'marital_status',  'current_age',  'pepfar_age_group', 'tri_pillar_age_group', 'paeds_adult_age_group', 
    'ts_name', 'ts_cell_phone_number', 'ts_second_name', 'ts_second_cell_phone_number',
    'weight','current_town', 'current_constituency', 'current_street', 'permanent_town', 'permanent_constituency', 'permanent_street', 
    'cbart_cargs_name', 'cbart_cargs_code','hiv_confirmation_date', 'hiv_confirmation_age', 'hiv_confirmatory_result_date', 'hiv_confirmatory_result_type', 'hiv_diagnosis_facility_id',
    'hiv_enrollment_date', 'hiv_enrollment_age','hiv_disclosure_enrollment_date', 'full_disclosure_date', 'hiv_enrollment_facility_id','arv_initiating_facility','art_start_date', 'months_since_art_start', 'months_since_art_restart','art_duration', 'has_ever_been_initiated_on_art', 
    'patient_status', 'who_stage', 'art_eligible_reason',  'first_visit_date', 'first_visit_age', 'first_visit_facility_id', 'last_visit_date', 'last_visit_age',  'last_next_visit_date', 'last_care_model', 'last_visit_facility_id', 
    'last_cc_treatment_type', 'last_cc_treatment_date', 'last_cc_results', 'last_oi', 'last_scheduled_visit_date', 'last_pregnancy_status', 'last_breast_feeding', 
    'last_oi_other', 'last_regimen_line', 'last_regimen', 'last_regimen_date', 'last_visit_duration', 'days_since_last_visit', 'years_since_last_visit', 'last_visit_month', 'last_visit_year',   'last_lmp', 'last_edd','last_tb_screen_result', 'first_viral_load_order_date', 'first_viral_load_result_date', 'first_viral_load_result_age',  
    'first_viral_load_result', 'first_viral_load_facility_id', 'first_viral_load_result_category', 'first_viral_load_result_status', 'last_viral_load_order_date', 
    'last_viral_load_result_date','last_viral_load_result_age', 'months_since_last_viral_load', 'last_viral_load_result', 'last_viral_load_facility_id', 'is_virally_suppressed', 'is_not_virally_suppressed', 'needs_vl_test', 'last_viral_load_result_status', 'reason_for_next_vl', 'next_vl_date', 
    'art_interruption_date', 'art_interruption_reason', 'art_interruption_reason_other' ,'art_restart_date',
    'tbt_start_date','tbt_regimen', 'tbt_expected_stop_date', 'tbt_actual_stop_date','tbt_facility_id', 'tbt_category',  'tbt_duration', 'tbt_registration_number', 
    'tbt_site', 'tbt_site_detail', 'tbt_outcome', 'tbt_outcome_date', 'tpt_facility_id', 'tpt_start_date', 'tpt_regimen', 'tpt_expected_stop_date', 'tpt_status','tpt_status_two_outcome','tpt_type',
    'tpt_actual_stop_date', 'tpt_stop_reason',  'tpt_adherance','death_date', 'iit_date', 'iit_duration', 'client_status', 'retention_status', 'tx_curr', 
    'is_last_viral_load_valid', 'last_viral_load_result_category',  'vl_eligibility', 'tpt_duration', 'tpt_duration_two',  'tpt_status_two', 'last_PBFW', 'last_PBFW_status','mmp_status', 'mmp_status_current', 'mmp_status_adult', 'mmp_status_paed', 'six_mmp_eligible', 'six_mmp_eligible_but_not_given', 'art_outcomes'    
]

# Reorder the columns in the DataFrame
fact_sentinel_event_df = fact_sentinel_event_df[new_column_order]

# Display the reordered DataFrame
fact_sentinel_event_df.head()

,client_id,region,district,facility_code,current_facility,id_facility_current,transfer_status,transfer_date,quantum_number,art_number,...,tpt_status_two,last_PBFW,last_PBFW_status,mmp_status,mmp_status_current,mmp_status_adult,mmp_status_paed,six_mmp_eligible,six_mmp_eligible_but_not_given,art_outcomes
0,F2E91406-846B-7D40-9544-C522FE691720,!Karas,!Nami≠Nûs,10221,Luderitz District Hospital,EF916F03-F628-4E36-9659-889E4443BCB9,In,0001-01-01,10221-1003-1,10221-1003-1,...,TPT completed,None,No,6+ MMP,None,Yes,None,Yes,No,VL Suppressed
1,EEB0C17D-193C-9A46-B0F3-447E59D74CB2,None,None,99999,Unknown,931237D1-CBAB-49DF-8844-67EB50DA522D,Out,2013-02-05,10221-1003-2,10221-1003-2,...,None,None,None,None,None,Yes,None,None,No,Needs VL Test
2,00BFC838-2A1C-AA41-9389-E480215EBE3D,None,None,99999,Unknown,931237D1-CBAB-49DF-8844-67EB50DA522D,Out,2008-09-01,10221-1103-3,10221-1103-3,...,None,None,No,None,None,Yes,None,None,No,Case Closed
3,6C708B5E-FF4F-064E-9597-230BFF374297,None,None,99999,Unknown,931237D1-CBAB-49DF-8844-67EB50DA522D,Out,2008-07-07,10221-1103-4,10221-1103-4,...,None,None,None,None,None,Yes,None,None,No,Case Closed
4,792BA050-2DAF-DA48-B918-786C5B751C76,Ohangwena,Engela,10389,Onamukulo Clinic,BF8397FF-AC48-4007-95D8-72CEAF62701B,In,0001-01-01,10221-1103-5,10221-1103-5,...,None,None,No,None,None,Yes,None,None,No,Needs VL Test


# Clean up column names

In [1236]:
def rename_columns(col):
    if '_id' in col:
        return '__' + col
    else:
        col = col.replace('_', ' ').title()
        col = col.replace('Tpt', 'TPT')
        col = col.replace('Tbt', 'TBT')
        col = col.replace('Hiv', 'HIV')
        col = col.replace('Art', 'ART')
        col = col.replace('Iit', 'IIT')
        col = col.replace('Mmp', 'MMP')
        col = col.replace('Pmtct', 'PMTCT')
        col = col.replace('Pbfw', 'PBFW')
        col = col.replace('Vl', 'Viral Load')
        col = col.replace('Tx Curr', 'TX Curr')
        col = col.replace('Oi', 'OI')
        col = col.replace('Lmp', 'LMP')
        col = col.replace('Edd', 'EDD')
        col = col.replace('Who', 'WHO')
        col = col.replace('Tb', 'TB')
        return col

fact_sentinel_event_df.columns = [rename_columns(col) for col in fact_sentinel_event_df.columns]

fact_sentinel_event_df.head()

,__client_id,Region,District,Facility Code,Current Facility,Id Facility Current,Transfer Status,Transfer Date,Quantum Number,ART Number,...,TPT Status Two,Last PBFW,Last PBFW Status,MMP Status,MMP Status Current,MMP Status Adult,MMP Status Paed,Six MMP Eligible,Six MMP Eligible But Not Given,ART Outcomes
0,F2E91406-846B-7D40-9544-C522FE691720,!Karas,!Nami≠Nûs,10221,Luderitz District Hospital,EF916F03-F628-4E36-9659-889E4443BCB9,In,0001-01-01,10221-1003-1,10221-1003-1,...,TPT completed,None,No,6+ MMP,None,Yes,None,Yes,No,VL Suppressed
1,EEB0C17D-193C-9A46-B0F3-447E59D74CB2,None,None,99999,Unknown,931237D1-CBAB-49DF-8844-67EB50DA522D,Out,2013-02-05,10221-1003-2,10221-1003-2,...,None,None,None,None,None,Yes,None,None,No,Needs VL Test
2,00BFC838-2A1C-AA41-9389-E480215EBE3D,None,None,99999,Unknown,931237D1-CBAB-49DF-8844-67EB50DA522D,Out,2008-09-01,10221-1103-3,10221-1103-3,...,None,None,No,None,None,Yes,None,None,No,Case Closed
3,6C708B5E-FF4F-064E-9597-230BFF374297,None,None,99999,Unknown,931237D1-CBAB-49DF-8844-67EB50DA522D,Out,2008-07-07,10221-1103-4,10221-1103-4,...,None,None,None,None,None,Yes,None,None,No,Case Closed
4,792BA050-2DAF-DA48-B918-786C5B751C76,Ohangwena,Engela,10389,Onamukulo Clinic,BF8397FF-AC48-4007-95D8-72CEAF62701B,In,0001-01-01,10221-1103-5,10221-1103-5,...,None,None,No,None,None,Yes,None,None,No,Needs VL Test


# Define Destination Database Connection

In [1237]:
# PostgreSQL server connection details
server = destination_environment.get("server")
database = destination_environment.get("database")
port = destination_environment.get("port")
username = destination_environment.get("username")
password = destination_environment.get("password")

# Create connection to the default database (e.g., postgres) to check if the target database exists
default_database = 'postgres'  # Usually, 'postgres' is used for administrative tasks
default_connection_url = f'postgresql://{username}:{password}@{server}:{port}/{default_database}'

# Set the isolation level to AUTOCOMMIT for creating the database
default_engine = create_engine(default_connection_url, isolation_level='AUTOCOMMIT')

# Query to check if the database exists
check_db_query = f"SELECT 1 FROM pg_database WHERE datname = '{database}'"

# Query to create the database if it doesn't exist
create_db_query = f"CREATE DATABASE {database}"

try:
    with default_engine.connect() as connection:
        # Check if the database exists
        result = connection.execute(text(check_db_query)).fetchone()

        if not result:
            # If the database does not exist, create it
            connection.execute(text(create_db_query))
            environment.log_message(f"Database '{database}' created successfully.")
        else:
            environment.log_message(f"Database '{database}' already exists.")
except Exception as e:
    print(f"Error occurred: {e}")

# Now connect to the target database
connection_url = f'postgresql://{username}:{password}@{server}:{port}/{database}'
engine = create_engine(connection_url)

[2024-11-20T18:01:12.516946] QE Data Task Running - INFO: Database 'duft' already exists.
QE Data Task Running >>>: Database 'duft' already exists.


# Function that checks if a table exists

In [1238]:


def create_and_insert_table(df, table_name, engine, schema_name):
    """
    This function checks if a schema and table exist in the given schema. If the schema does not exist,
    it creates the schema. If the table does not exist, it creates the table and inserts data from the
    provided DataFrame. If the table exists, it replaces the data.

    Parameters:
    df (pandas.DataFrame): The DataFrame containing data to be inserted.
    table_name (str): The name of the table to create or replace.
    engine (sqlalchemy.engine.Engine): The SQLAlchemy engine to connect to the PostgreSQL database.
    schema_name (str): The schema name where the table is located. Default is 'public'.
    """
    
    # SQL query to check if the schema exists
    check_schema_query = f"""
    SELECT EXISTS (
        SELECT 1 FROM information_schema.schemata 
        WHERE schema_name = '{schema_name}'
    );
    """
    
    # SQL query to check if the table exists
    check_table_query = f"""
    SELECT EXISTS (
        SELECT FROM information_schema.tables 
        WHERE table_schema = '{schema_name}' 
        AND table_name = '{table_name}'
    );
    """
    
    try:
        with engine.connect() as connection:
            # Begin a new transaction
            with connection.begin() as transaction:
                # Check if the schema exists
                schema_exists = connection.execute(text(check_schema_query)).fetchone()

                if not schema_exists[0]:  # If the schema doesn't exist
                    environment.log_message(f"Schema '{schema_name}' does not exist. Creating it...")
                    # Create the schema
                    connection.execute(text(f"CREATE SCHEMA IF NOT EXISTS {schema_name};"))
                    environment.log_message(f"Schema '{schema_name}' created successfully.")

            # Check if the table exists (in a separate transaction to avoid conflicts)
            with connection.begin() as transaction:
                table_exists = connection.execute(text(check_table_query)).fetchone()

                if not table_exists[0]:  # If the table doesn't exist
                    environment.log_message(f"Table '{table_name}' does not exist in schema '{schema_name}'. Creating it...")
                    # Automatically create the table based on the DataFrame structure
                    df.to_sql(table_name, con=engine, if_exists='replace', index=False, schema=schema_name)
                    environment.log_message(f"Table '{table_name}' created and data inserted successfully.")
                else:
                    # If the table exists, replace the data
                    df.to_sql(table_name, con=engine, if_exists='replace', index=False, schema=schema_name)
                    environment.log_message(f"Data replaced successfully in table '{table_name}'.")

    except Exception as e:
        print(f"Error occurred: {e}")




# Write dim age group to table

In [1239]:
table_name = 'dim_age_group'
schema_name = 'analysis'

create_and_insert_table(dim_age_group_df, table_name, engine, schema_name)

dim_age_group_df.head()

[2024-11-20T18:01:12.623263] QE Data Task Running - INFO: Data replaced successfully in table 'dim_age_group'.
QE Data Task Running >>>: Data replaced successfully in table 'dim_age_group'.


,age,pepfar_age_group,paeds_adult_age_group,tri_pillar_age_group_eighteen,tri_pillar_age_group,tri_pillar_age_group_val,paeds_adult_age_group_val,pepfar_age_group_val
0,0,0-4,Pediatric,0-18,0-19,1,2,1
1,1,0-4,Pediatric,0-18,0-19,1,2,1
2,2,0-4,Pediatric,0-18,0-19,1,2,1
3,3,0-4,Pediatric,0-18,0-19,1,2,1
4,4,5-9,Pediatric,0-18,0-19,1,2,2


# Drop View

In [1240]:
drop_views_sql = '''
DROP VIEW IF EXISTS "analysis"."score_card";
'''

try:
    with engine.connect() as conn:
        with conn.begin():
            conn.execute(text(drop_views_sql))
            print("View dropped successfully.")
except Exception as e:
    print(f"An error occurred: {e}")

View dropped successfully.


# Write sentinel event to table

In [1241]:
table_name = 'fact_sentinel_event'
schema_name = 'analysis'

create_and_insert_table(fact_sentinel_event_df, table_name, engine, schema_name)

fact_sentinel_event_df.head()

[2024-11-20T18:01:13.916906] QE Data Task Running - INFO: Data replaced successfully in table 'fact_sentinel_event'.
QE Data Task Running >>>: Data replaced successfully in table 'fact_sentinel_event'.


,__client_id,Region,District,Facility Code,Current Facility,Id Facility Current,Transfer Status,Transfer Date,Quantum Number,ART Number,...,TPT Status Two,Last PBFW,Last PBFW Status,MMP Status,MMP Status Current,MMP Status Adult,MMP Status Paed,Six MMP Eligible,Six MMP Eligible But Not Given,ART Outcomes
0,F2E91406-846B-7D40-9544-C522FE691720,!Karas,!Nami≠Nûs,10221,Luderitz District Hospital,EF916F03-F628-4E36-9659-889E4443BCB9,In,0001-01-01,10221-1003-1,10221-1003-1,...,TPT completed,None,No,6+ MMP,None,Yes,None,Yes,No,VL Suppressed
1,EEB0C17D-193C-9A46-B0F3-447E59D74CB2,None,None,99999,Unknown,931237D1-CBAB-49DF-8844-67EB50DA522D,Out,2013-02-05,10221-1003-2,10221-1003-2,...,None,None,None,None,None,Yes,None,None,No,Needs VL Test
2,00BFC838-2A1C-AA41-9389-E480215EBE3D,None,None,99999,Unknown,931237D1-CBAB-49DF-8844-67EB50DA522D,Out,2008-09-01,10221-1103-3,10221-1103-3,...,None,None,No,None,None,Yes,None,None,No,Case Closed
3,6C708B5E-FF4F-064E-9597-230BFF374297,None,None,99999,Unknown,931237D1-CBAB-49DF-8844-67EB50DA522D,Out,2008-07-07,10221-1103-4,10221-1103-4,...,None,None,None,None,None,Yes,None,None,No,Case Closed
4,792BA050-2DAF-DA48-B918-786C5B751C76,Ohangwena,Engela,10389,Onamukulo Clinic,BF8397FF-AC48-4007-95D8-72CEAF62701B,In,0001-01-01,10221-1103-5,10221-1103-5,...,None,None,No,None,None,Yes,None,None,No,Needs VL Test


# Get current facility

In [1242]:
facility_counts = dim_pat_df['current_facility'].value_counts()
current_facility = facility_counts.idxmax()

# Add last refresh date

In [1243]:
data = {
    'last_refresh_date': [datetime.today().strftime('%Y-%m-%d')],
    'facility_name': current_facility
}

last_refresh_date_df = pd.DataFrame(data)

print(last_refresh_date_df)

  last_refresh_date               facility_name
0        2024-11-20  Luderitz District Hospital


# Write last refresh to table 

In [1244]:
table_name = 'dim_last_refresh'
schema_name = 'analysis'


create_and_insert_table(last_refresh_date_df, table_name, engine, schema_name)

last_refresh_date_df.head()

[2024-11-20T18:01:13.949044] QE Data Task Running - INFO: Data replaced successfully in table 'dim_last_refresh'.
QE Data Task Running >>>: Data replaced successfully in table 'dim_last_refresh'.


,last_refresh_date,facility_name
0,2024-11-20,Luderitz District Hospital


# Create Score Card View

In [1245]:
view_score_card_creation_query = """
CREATE VIEW analysis.score_card AS
 WITH ordered_data AS (
         SELECT '6+ MMP'::text AS indicator,
            sum(
                CASE
                    WHEN fact_sentinel_event."TX Curr" = 'Yes'::text AND fact_sentinel_event."MMP Status" = '6+ MMP'::text THEN 1
                    ELSE 0
                END)::numeric * 100.0 / count(
                CASE
                    WHEN fact_sentinel_event."TX Curr" = 'Yes'::text THEN 1
                    ELSE NULL::integer
                END)::numeric AS score,
            1 AS order_col
           FROM analysis.fact_sentinel_event
        UNION ALL
         SELECT 'TPT'::text AS indicator,
            sum(
                CASE
                    WHEN fact_sentinel_event."TX Curr" = 'Yes'::text AND fact_sentinel_event."TPT Status Two" = 'TPT completed'::text THEN 1
                    ELSE 0
                END)::numeric * 100.0 / count(
                CASE
                    WHEN fact_sentinel_event."TX Curr" = 'Yes'::text THEN 1
                    ELSE NULL::integer
                END)::numeric AS score,
            2 AS order_col
           FROM analysis.fact_sentinel_event
        UNION ALL
         SELECT 'Retention'::text AS indicator,
            sum(
                CASE
                    WHEN (fact_sentinel_event."Retention Status" = ANY (ARRAY['In Care'::text, 'Missed Appointments'::text])) THEN 1
                    ELSE 0
                END)::numeric * 100.0 / count(
                CASE
                    WHEN fact_sentinel_event."Retention Status" = ANY (ARRAY['In Care'::text, 'Missed Appointments'::text, 'Treatment Interruptions'::text]) THEN 1
                    ELSE NULL::integer
                END)::numeric AS score,
            3 AS order_col
           FROM analysis.fact_sentinel_event
        UNION ALL
         SELECT 'VLM'::text AS indicator,
            sum(
                CASE
                    WHEN fact_sentinel_event."TX Curr" = 'Yes'::text AND (fact_sentinel_event."Viral Load Eligibility" = ANY (ARRAY['Monitored per Guidelines'::text, 'Slightly Delayed'::text])) THEN 1
                    ELSE 0
                END)::numeric * 100.0 / count(
                CASE
                    WHEN fact_sentinel_event."TX Curr" = 'Yes'::text THEN 1
                    ELSE NULL::integer
                END)::numeric AS score,
            4 AS order_col
           FROM analysis.fact_sentinel_event
        UNION ALL
         SELECT 'VLS (VL<40 CPS/ML)'::text AS indicator,
            sum(
                CASE
                    WHEN fact_sentinel_event."TX Curr" = 'Yes'::text AND fact_sentinel_event."Is Virally Suppressed" = 'Yes'::text THEN 1
                    ELSE 0
                END)::numeric * 100.0 / count(
                CASE
                    WHEN fact_sentinel_event."TX Curr" = 'Yes'::text AND fact_sentinel_event."Is Last Viral Load Valid" = 'Yes'::text THEN 1
                    ELSE NULL::integer
                END)::numeric AS score,
            5 AS order_col
           FROM analysis.fact_sentinel_event
        )
 SELECT indicator,
    round(score, 2) AS score,
        CASE
            WHEN indicator = '6+ MMP'::text AND score < 50::numeric THEN 'Bad'::text
            WHEN indicator = '6+ MMP'::text AND score >= 50::numeric AND score < 75::numeric THEN 'Average'::text
            WHEN indicator = '6+ MMP'::text AND score >= 75::numeric THEN 'Good'::text
            WHEN indicator = 'TPT'::text AND score < 85::numeric THEN 'Bad'::text
            WHEN indicator = 'TPT'::text AND score >= 85::numeric AND score < 95::numeric THEN 'Average'::text
            WHEN indicator = 'TPT'::text AND score >= 95::numeric THEN 'Good'::text
            WHEN indicator = 'Retention'::text AND score < 90::numeric THEN 'Bad'::text
            WHEN indicator = 'Retention'::text AND score >= 90::numeric AND score < 95::numeric THEN 'Average'::text
            WHEN indicator = 'Retention'::text AND score >= 95::numeric THEN 'Good'::text
            WHEN indicator = 'VLM'::text AND score < 80::numeric THEN 'Bad'::text
            WHEN indicator = 'VLM'::text AND score >= 80::numeric AND score < 90::numeric THEN 'Average'::text
            WHEN indicator = 'VLM'::text AND score >= 90::numeric THEN 'Good'::text
            WHEN indicator = 'VLS (VL<40 CPS/ML)'::text AND score < 85::numeric THEN 'Bad'::text
            WHEN indicator = 'VLS (VL<40 CPS/ML)'::text AND score >= 85::numeric AND score < 93::numeric THEN 'Average'::text
            WHEN indicator = 'VLS (VL<40 CPS/ML)'::text AND score >= 93::numeric THEN 'Good'::text
            ELSE 'Unknown'::text
        END AS color
   FROM ordered_data
  ORDER BY order_col;
"""

try:
    with engine.connect() as conn:
        with conn.begin():
            conn.execute(text(view_score_card_creation_query))
            environment.log_message("Score Card view created successfully.")
except Exception as e:
    print(f"An error occurred: {e}")

[2024-11-20T18:01:13.958992] QE Data Task Running - INFO: Score Card view created successfully.
QE Data Task Running >>>: Score Card view created successfully.


# Drop dataframes

In [1246]:
del dim_pat_df
del fact_hiv_diagnosis_df
del fact_hiv_enrolment_df
del fact_first_visit_df
del fact_last_visit_df
del fact_visits_df
del fact_lab_df
del fact_tpt_df
del fact_meas_df
del fact_first_viral_load_tests_df
del fact_last_viral_load_tests_df
del fact_tbt_df
del fact_tsfr_df

environment.log_message(f'Finished writing data to database')

[2024-11-20T18:01:13.986308] QE Data Task Running - INFO: Finished writing data to database
QE Data Task Running >>>: Finished writing data to database


# Log end time

In [1247]:
pd.Timestamp.today()

Timestamp('2024-11-20 18:01:13.991157')